In [1]:
from math import sqrt
import copy
import  traceback
import shutil

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image

In [2]:
def load_scan(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in filter(lambda x: x.endswith('.dcm'), os.listdir(path))]
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices


def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16),
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    # image[image <= -2000] = 0

    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        height, width = image[slice_number].shape[0], image[slice_number].shape[1]
        
        image[slice_number][image[slice_number]<=-2000] = image[slice_number, int(height*0.05), int(width*0.5)]
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
        image[slice_number] += np.int16(intercept)

    return np.array(image, dtype=np.int16)


class Elem():
    def __init__(self, key, contour, t, out_of_range):
        self.root = key
        self.end = key
        self.contours = [contour]
        self.keys = [key]
        self.t = t
        self.out_of_range = out_of_range
        
    def __len__(self):
        return len(self.contours)
    
    def append(self, key, contour, out_of_range):
        self.end = key
        self.contours.append(contour)
        self.keys.append(key)
        self.out_of_range = self.out_of_range or out_of_range
        
    def get_bboxes(self):
        self.circles = [cv2.minEnclosingCircle(c) for c in self.contours]
        def process(circle):
            cx, cy, r = round(circle[0][0]), round(circle[0][1]), circle[1]
            return cx - int(1.5*r), cy - int(1.5*r), cx + int(1.5*r), cy + int(1.5*r)
            
        self.bboxes = [process(c) for c in self.circles]
        
    def __str__(self):
        return str(self.root)+'->'+str(self.end)+':'+str(self.keys)
    
    def get_prev_area(self):
        return cv2.contourArea(self.contours[-2])
    
    @staticmethod
    def calc_ratio(proportion, kind=0, lower=0.08, upper=0.12):
        if proportion >= upper:
            return 1 if kind == 0 else 1 if kind == 1 else 1.2
        elif proportion <= lower:
            return 3 if kind == 0 else 8 if kind == 1 else 1.5
        else:
            if kind == 0:
                return 2*sqrt((upper-proportion)/(upper-lower))+1 
            elif kind == 1:
                return (upper-proportion)/(upper-lower)*7+1
            else:
                return 0.3*sqrt((upper-proportion)/(upper-lower))+1.2
    
    def post_process(self, patient_pixels, start_x, start_y):       
        self.post_bboxes = []
        img_height, img_width = patient_pixels.shape[1:3]
        start, end = self.root[0], self.end[0]+1
        for i in range(start, end):
            #cx, cy, r = round(self.circles[i-start][0][0]+start_x), round(self.circles[i-start][0][1]+start_y), int(self.circles[i-start][1])
            cx = (self.bboxes[i-start][2]+self.bboxes[i-start][0])//2 + start_x
            cy = (self.bboxes[i-start][3]+self.bboxes[i-start][1])//2 + start_y
            r = int((self.bboxes[i-start][2]-self.bboxes[i-start][0]+1)/3)
            sx, sy = max(0, int(cx-4.5*r)), max(0, int(cy-4.5*r))
            ex, ey = min(img_width, int(cx+4.5*r+1)), min(img_height, int(cy+4.5*r+1))
            gray = patient_pixels[i, sy:ey, sx:ex].astype(np.float32)
            gray = np.clip(gray, 0, 100)
            gray = (gray-gray.min())/(gray.max()-gray.min())*255
            gray = gray.astype(np.uint8)
            gray = cv2.medianBlur(gray, 7)
            # gray = cv2.GaussianBlur(gray, (7,7), 2)
            lap = cv2.Laplacian(gray, cv2.CV_16S)
            gray = gray.astype(np.int16) - lap
            gray = np.clip(gray, 0, 255).astype(np.uint8)
            min_edge = min(img_height, img_width)
            proportion = r*3 / min_edge
            fc = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1, minDist=min(*gray.shape)//2, param1=200, param2=19, minRadius=r, maxRadius=int(min(min_edge*0.055,r*self.calc_ratio(proportion)+1)))
            if fc is None:
                #self.post_bboxes.append((self.bboxes[i-start][0]+start_x, self.bboxes[i-start][1]+start_y, self.bboxes[i-start][2]+start_x, self.bboxes[i-start][3]+start_y))
                ratio = self.calc_ratio(proportion, kind=2)
                m, n, p, q = cx - int(ratio*r), cy - int(ratio*r), cx + int(ratio*r), cy + int(ratio*r)
                self.post_bboxes.append((m, n, p, q))
#                 if 220 <= i <= 260:
#                     print(f'{i}:put origin')
            else:
                if len(fc) != 1:
                    raise Exception(f'len(fc) == {len(fc)}')
                fc = fc[0]
#                 if 220 <= i <= 260:
#                     print(f'{i}:{fc}')
                for circle in fc:
                    fx, fy, fr = round(circle[0]+sx), round(circle[1]+sy), int(circle[2])
                    assert fr >= r
                    dis = sqrt((fx-cx)**2+(fy-cy)**2)
#                     if 220 <= i <= 260:
#                         print(f'{i}:fr:{fr},r:{r}|{0.5*(fr-r)}<{dis}<{fr - r/self.calc_ratio(proportion, 1)} and {(fr-r)/r}>0.1')
                    if 0.5*(fr-r) < dis < fr - r/self.calc_ratio(proportion, 1) and (fr-r)/r > 0.1:
                        m, n, p, q = fx-int(1.3*fr), fy-int(1.3*fr), fx+int(1.3*fr), fy+int(1.3*fr)
                        self.post_bboxes.append((m, n, p, q))
#                         if 220 <= i <= 260:
#                             print(f'{i}:put new')
                        break
                else:
                    #self.post_bboxes.append((self.bboxes[i-start][0]+start_x, self.bboxes[i-start][1]+start_y, self.bboxes[i-start][2]+start_x, self.bboxes[i-start][3]+start_y))
                    ratio = self.calc_ratio(proportion, kind=2)
                    m, n, p, q = cx - int(ratio*r), cy - int(ratio*r), cx + int(ratio*r), cy + int(ratio*r)
                    self.post_bboxes.append((m, n, p, q))
#                     if 220 <= i <= 260:
#                         print(f'{i}:put origin')
                    
    def blur_bboxes(self, kind=0):
        if kind == 0:
            bboxes = self.bboxes
        else:
            bboxes = self.post_bboxes
            
        xs = [(x1+x2)//2 for x1, y1, x2, y2 in bboxes]
        ys = [(y1+y2)//2 for x1, y1, x2, y2 in bboxes]
        ws = [x2-x1+1 for x1, y1, x2, y2 in bboxes]
        ksize, thresh, dis_thresh = 8, 0.20, 0.20
        if len(bboxes) < 300:
            ksize, thresh, dis_thresh = 5, 0.25, 0.25
        for i in range(ksize, len(bboxes)-ksize):
            mxs = sorted(xs[i-ksize: i+ksize+1])
            mys = sorted(ys[i-ksize: i+ksize+1])
            mws = sorted(ws[i-ksize: i+ksize+1])
            cx, cy, cw = mxs[ksize], mys[ksize], mws[ksize]
            if abs(cw-ws[i]) > cw * thresh:
                r = cw//2
                bboxes[i] = (cx-r, cy-r, cx+r, cy+r)
            elif kind == 1:
                dis = sqrt((cx-xs[i])**2+(cy-ys[i])**2)
                if dis > cw * dis_thresh:
                    r = cw//2
                    bboxes[i] = (cx-r, cy-r, cx+r, cy+r)

In [3]:
def draw_bbox(slices, path, start_x, start_y, branch, branch_start, branch_end, crown_start, aorta_end, save_path):
    sl = save_path.split('/')
    name = sl[-3] if sl[-1] != '' else sl[-4]
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    if not os.path.exists(os.path.join(save_path, 'picture')):
        os.mkdir(os.path.join(save_path, 'picture'))
    start = path.root[0]
    for i, s in enumerate(slices):
        img = s.pixel_array
        intercept = slices[i].RescaleIntercept
        slope = slices[i].RescaleSlope
        if crown_start <= i < aorta_end:
            x1, y1 = path.bboxes[i-start][0] + start_x, path.bboxes[i-start][1] + start_y
            x2, y2 = path.bboxes[i-start][2] + start_x, path.bboxes[i-start][3] + start_y
            picture = img[y1:y2+1, x1:x2+1].astype(np.int16)
            if slope != 1:
                picture = (slope * picture.astype(np.float64)).astype(np.int16)
            picture += np.int16(intercept)
            picture = np.clip(picture, -100, 500)
            picture = ((picture+100)/600*255).astype(np.uint8)
            picture = Image.fromarray(picture)
            picture.save(os.path.join(save_path, f'picture/1_{name}_{i}.png'))
        if branch_start <= i < branch_end:
            x1, y1 = branch.bboxes[i-start][0] + start_x, branch.bboxes[i-start][1] + start_y
            x2, y2 = branch.bboxes[i-start][2] + start_x, branch.bboxes[i-start][3] + start_y
            picture = img[y1:y2+1, x1:x2+1].astype(np.int16)
            if slope != 1:
                picture = (slope * picture.astype(np.float64)).astype(np.int16)
            picture += np.int16(intercept)
            picture = np.clip(picture, -100, 500)
            picture = ((picture+100)/600*255).astype(np.uint8)
            picture = Image.fromarray(picture)
            picture.save(os.path.join(save_path, f'picture/2_{name}_{i}.png'))
        if path.root[0] <= i <= path.end[0]:
            x1, y1 = path.bboxes[i-start][0] + start_x, path.bboxes[i-start][1] + start_y
            x2, y2 = path.bboxes[i-start][2] + start_x, path.bboxes[i-start][3] + start_y
            cv2.rectangle(img, (x1, y1), (x2, y2), 3000, 1)
            if branch_start <= i < branch_end:
                x1, y1 = branch.bboxes[i-start][0] + start_x, branch.bboxes[i-start][1] + start_y
                x2, y2 = branch.bboxes[i-start][2] + start_x, branch.bboxes[i-start][3] + start_y
                cv2.rectangle(img, (x1, y1), (x2, y2), 3000, 1)
        s.PixelData = pydicom.encaps.encapsulate([img.tobytes()])
        s.file_meta.TransferSyntaxUID = '1.2.840.10008.1.2.1'
        s.save_as(os.path.join(save_path, f'{i}.dcm'), write_like_original=False)
        
def draw_post_bbox(slices, path, branch, branch_start, branch_end, crown_start, aorta_end, save_path):
    sl = save_path.split('/')
    name = sl[-3] if sl[-1] != '' else sl[-4]
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    if not os.path.exists(os.path.join(save_path, 'picture')):
        os.mkdir(os.path.join(save_path, 'picture'))
    start = path.root[0]
    for i, s in enumerate(slices):
        img = s.pixel_array
        intercept = slices[i].RescaleIntercept
        slope = slices[i].RescaleSlope
        if crown_start <= i < aorta_end:
            x1, y1 = path.post_bboxes[i-start][0], path.post_bboxes[i-start][1]
            x2, y2 = path.post_bboxes[i-start][2], path.post_bboxes[i-start][3]
            picture = img[y1:y2+1, x1:x2+1].astype(np.int16)
            if slope != 1:
                picture = (slope * picture.astype(np.float64)).astype(np.int16)
            picture += np.int16(intercept)
            picture = np.clip(picture, -100, 500)
            picture = ((picture+100)/600*255).astype(np.uint8)
            picture = Image.fromarray(picture)
            picture.save(os.path.join(save_path, f'picture/1_{name}_{i}.png'))
        if branch_start <= i < branch_end:
            x1, y1 = branch.post_bboxes[i-start][0], branch.post_bboxes[i-start][1]
            x2, y2 = branch.post_bboxes[i-start][2], branch.post_bboxes[i-start][3]
            picture = img[y1:y2+1, x1:x2+1].astype(np.int16)
            if slope != 1:
                picture = (slope * picture.astype(np.float64)).astype(np.int16)
            picture += np.int16(intercept)
            picture = np.clip(picture, -100, 500)
            picture = ((picture+100)/600*255).astype(np.uint8)
            picture = Image.fromarray(picture)
            picture.save(os.path.join(save_path, f'picture/2_{name}_{i}.png'))
        if path.root[0] <= i <= path.end[0]:
            x1, y1 = path.post_bboxes[i-start][0], path.post_bboxes[i-start][1]
            x2, y2 = path.post_bboxes[i-start][2], path.post_bboxes[i-start][3]
            cv2.rectangle(img, (x1, y1), (x2, y2), 3000, 1)
            if branch_start <= i < branch_end:
                x1, y1 = branch.post_bboxes[i-start][0], branch.post_bboxes[i-start][1]
                x2, y2 = branch.post_bboxes[i-start][2], branch.post_bboxes[i-start][3]
                cv2.rectangle(img, (x1, y1), (x2, y2), 3000, 1)
        s.PixelData = pydicom.encaps.encapsulate([img.tobytes()])
        s.file_meta.TransferSyntaxUID = '1.2.840.10008.1.2.1'
        s.save_as(os.path.join(save_path, f'{i}.dcm'), write_like_original=False)

def adjust_gamma(img, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    new_img = np.empty(img.shape)
    new_img = cv2.LUT(img, table)
    return new_img

def get_intersection(origin, first, second):
    zero1, zero2 = np.zeros_like(origin), np.zeros_like(origin)
    cv2.fillPoly(zero1, [first], 125)
    cv2.fillPoly(zero2, [second], 130)
    inter = zero1 + zero2
    inter[inter<255] = 0
    contours, _ = cv2.findContours(inter, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    inter_area = 0
    for contour in contours:
        inter_area += cv2.contourArea(contour)
    first_area = cv2.contourArea(first)
    second_area = cv2.contourArea(second)
    if first_area == 0 or second_area == 0:
        return 0
    assert inter_area <= first_area and inter_area <= second_area
    return max(inter_area/first_area, inter_area/second_area)

def set_window00(image, w_center, w_width, crown_start, branch_start):
    image_copy = image.copy()
    image_copy = np.clip(image_copy, w_center-int(w_width/2), w_center+int(w_width/2))
    image_copy = image_copy.astype(np.float32)
    image_copy = ((image_copy - image_copy.min())/(image_copy.max() - image_copy.min())*255).astype(np.uint8)
    for slice_number in range(len(image)):
        min_l = (int(min(image_copy[slice_number].shape[0]*0.3, image_copy[slice_number].shape[1]*0.3))-1) // 2 * 2 + 1
        
        image_copy[slice_number] = cv2.GaussianBlur(image_copy[slice_number], (3,3), 1)
        lap = cv2.Laplacian(image_copy[slice_number], cv2.CV_16S)
        image_copy[slice_number] = np.clip(image_copy[slice_number].astype(np.int16) - lap, 0, 255).astype(np.uint8)
        image_copy[slice_number] = cv2.adaptiveThreshold(image_copy[slice_number], 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, min_l, 0)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        if crown_start <= slice_number <= branch_start+1:
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=2)
            image_copy[slice_number] = cv2.erode(image_copy[slice_number], kernel, iterations=7)
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=3)
        else:
            image_copy[slice_number] = cv2.erode(image_copy[slice_number], kernel, iterations=5)
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=3)
    return image_copy

def set_window01(image, w_center, w_width, crown_start, branch_start):
    image_copy = image.copy()
    image_copy = np.clip(image_copy, w_center-int(w_width/2), w_center+int(w_width/2))
    image_copy = image_copy.astype(np.float32)
    image_copy = ((image_copy - image_copy.min())/(image_copy.max() - image_copy.min())*255).astype(np.uint8)
    for slice_number in range(len(image)):
        min_l = (int(min(image_copy[slice_number].shape[0]*0.3, image_copy[slice_number].shape[1]*0.3))-1) // 2 * 2 + 1
        
        image_copy[slice_number] = cv2.medianBlur(image_copy[slice_number], 3)
        lap = cv2.Laplacian(image_copy[slice_number], cv2.CV_16S)
        image_copy[slice_number] = np.clip(image_copy[slice_number].astype(np.int16) - lap, 0, 255).astype(np.uint8)
        image_copy[slice_number] = cv2.adaptiveThreshold(image_copy[slice_number], 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, min_l, 0)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        if crown_start <= slice_number <= branch_start+1:
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=2)
            image_copy[slice_number] = cv2.erode(image_copy[slice_number], kernel, iterations=7)
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=3)
        else:
            image_copy[slice_number] = cv2.erode(image_copy[slice_number], kernel, iterations=5)
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=3)
    return image_copy

def set_window02(image, w_center, w_width, crown_start, branch_start):
    image_copy = image.copy()
    image_copy = np.clip(image_copy, w_center-int(w_width/2), w_center+int(w_width/2))
    image_copy = image_copy.astype(np.float32)
    image_copy = ((image_copy - image_copy.min())/(image_copy.max() - image_copy.min())*255).astype(np.uint8)
    for slice_number in range(len(image)):
        min_l = (int(min(image_copy[slice_number].shape[0]*0.3, image_copy[slice_number].shape[1]*0.3))-1) // 2 * 2 + 1
        
        image_copy[slice_number] = cv2.GaussianBlur(image_copy[slice_number], (3,3), 1)
        image_copy[slice_number] = cv2.adaptiveThreshold(image_copy[slice_number], 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, min_l, 0)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        if crown_start <= slice_number <= branch_start+1:
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=2)
            image_copy[slice_number] = cv2.erode(image_copy[slice_number], kernel, iterations=7)
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=3)
        else:
            image_copy[slice_number] = cv2.erode(image_copy[slice_number], kernel, iterations=5)
            image_copy[slice_number] = cv2.dilate(image_copy[slice_number], kernel, iterations=3)
    return image_copy

set_window_list = [set_window00, set_window01, set_window02]

In [ ]:
input_folder_list = []
root_path = '/nfs3-p1/zsxm/dataset/aorta/'
for path in os.listdir(root_path):
    if not os.path.isdir(os.path.join(root_path, path)):
        continue
    if not os.path.exists(os.path.join(root_path, path, '2/')):
        continue
    else:
        input_folder_list.append(os.path.join(root_path, path, '2/'))

In [4]:
# input_folder_list = ['/nfs3-p1/zsxm/dataset/aorta_CTA/chenxiu/img/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/dingxing/img/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/huanggaoqiang/img/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/louhangqing/img/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/wangxiaowei/yes/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/xiangxijiao/img/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/yangxiangqin/yes/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/zhaoqifeng/img/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/zhujingchun/img/',
#                      '/nfs3-p1/zsxm/dataset/aorta_CTA/zuochun/img/',
#                      '/nfs3-p1/zsxm/dataset/aortic_dissection/CTA1(+)/+C/',
#                      '/nfs3-p1/zsxm/dataset/aortic_dissection/CTA2(+)/+C/',
#                      '/nfs3-p1/zsxm/dataset/aortic_dissection/CTA2(-)/+c/',
#                      '/nfs3-p1/zsxm/dataset/aortic_dissection/CTA3(+)/+C/',
#                     ]
input_folder_list = [
    "/nfs3-p1/zsxm/dataset/aorta/chuguangzhou-Im46-111/2/",
#     "/nfs3-p1/zsxm/dataset/aorta/modehe-Im34-89/2/",
#     "/nfs3-p1/zsxm/dataset/aorta/yuliyao-Im25-31/2/",
#     "/nfs3-p1/zsxm/dataset/aorta/jiangquanfeng-Im 50-107/2/",
#     "/nfs3-p1/zsxm/dataset/aorta/shuzhiming-Im28-80/2/",
#     "/nfs3-p1/zsxm/dataset/aorta_CTA/huanggaoqiang/img/",
#     "/nfs3-p1/zsxm/dataset/aorta_CTA/xiangxijiao/img/",
#     '/nfs3-p1/zsxm/dataset/aorta/shenjiaren-Im35-136/2/',
#     '/nfs3-p1/zsxm/dataset/aorta/wanggaosheng-Im24-57/2/',
#     '/nfs3-p1/zsxm/dataset/aorta/wangzhongwen-Im36-80/2/',
#     '/nfs3-p1/zsxm/dataset/aorta/xujinyang-Im15-30/2/',
#     '/nfs3-p1/zsxm/dataset/aorta/zhangxinliang-Im16-24/2/',
#     '/nfs3-p1/zsxm/dataset/aorta/fengyonghua-Im21-89/2/',
]

In [9]:
dst_path = '/nfs3-p1/zsxm/dataset/aorta_temp/'
if not os.path.exists(dst_path):
    os.mkdir(dst_path)
for p in input_folder_list:
    sl = p.split('/')
    name = sl[-2] if sl[-1] != '' else sl[-3]
    if os.path.exists(os.path.join(dst_path, name)):
        print(f"remove {os.path.join(dst_path, name)}")
        shutil.rmtree(os.path.join(dst_path, name))
        
    print(f"Copy {os.path.join(p, 'save_processed')} To {os.path.join(dst_path, name)}")
    shutil.copytree(os.path.join(p, 'save_processed'), os.path.join(dst_path, name))

Copy /nfs3-p1/zsxm/dataset/aorta/chuguangzhou-Im46-111/2/save_processed To /nfs3-p1/zsxm/dataset/aorta_temp/chuguangzhou-Im46-111


FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta/chuguangzhou-Im46-111/2/save_processed'

In [23]:
for p in input_folder_list:
    if os.path.exists(os.path.join(p, 'save')):
        shutil.rmtree(os.path.join(p, 'save'))
        print(f"remove {os.path.join(p, 'save')}")
    if os.path.exists(os.path.join(p, 'save_post')):
        shutil.rmtree(os.path.join(p, 'save_post'))
        print(f"remove {os.path.join(p, 'save_post')}")
    if os.path.exists(os.path.join(p, 'save_processed')):
        shutil.rmtree(os.path.join(p, 'save_processed'))
        print(f"remove {os.path.join(p, 'save_processed')}")
    if os.path.exists(os.path.join(p, 'save_raw')):
        shutil.rmtree(os.path.join(p, 'save_raw'))
        print(f"remove {os.path.join(p, 'save_raw')}")
    if os.path.exists(os.path.join(p, 'save_raw_blur')):
        shutil.rmtree(os.path.join(p, 'save_raw_blur'))
        print(f"remove {os.path.join(p, 'save_raw_blur')}")

remove /nfs3-p1/zsxm/dataset/aorta/chuguangzhou-Im46-111/2/save_raw


In [32]:
for p in range(len(input_folder_list)):
    print(f'******Processing: {input_folder_list[p]}  ******')
    patient = load_scan(input_folder_list[p])
    patient_pixels = get_pixels_hu(patient)
    img_height, img_width = patient_pixels.shape[1:3]
    print(f'sequence length: {patient_pixels.shape[0]}, height: {img_height}, width: {img_width}')
    spine_x, spine_y = 0, 0
    
    patient_sum = np.sum(patient_pixels, axis=0)
    patient_sum = patient_sum.astype(np.float32)
    max_val = patient_sum.max()
    times = 0.6
    while times > 0:
        thr, patient_thr = cv2.threshold(patient_sum, max_val*times, 255, cv2.THRESH_BINARY)
        patient_thr = patient_thr.astype(np.uint8)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        patient_thr = cv2.erode(patient_thr, kernel, iterations=2)
        sum_contours, _ = cv2.findContours(patient_thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        sum_contours_list = [(cv2.contourArea(contour), contour) for contour in sum_contours]
        sum_contours_list.sort(key=lambda x: x[0], reverse=True)
        if len(sum_contours_list) == 0 \
        or len(sum_contours_list) > 1 and sum_contours_list[0][0] < 2 * sum_contours_list[1][0] \
        or sum_contours_list[0][0] < img_height*img_width*0.002:
            times -= 0.05
            continue
        find_flag = False
        for area, contour in sum_contours_list:
            print(f'find spine area: {area}')
            x, y, w, h = cv2.boundingRect(contour)
            cx, cy = x+w//2, y+h//2
            if img_height/2 < cy < img_height*0.75 and img_width*0.375 < cx < img_width*0.625:
                spine_x, spine_y = cx, cy
                find_flag = True
                break
        if find_flag:
            print(f'times:{times}')
            break
        times -= 0.05

    if spine_x == 0 or spine_y == 0:
        #raise Exception(f'spine_x:{spine_x}, spine_y:{spine_y}')
        print(f"spine_x:{spine_x}, spine_y:{spine_y}, can't find spine")
        continue
        
    with open(os.path.join(input_folder_list[p], 'branch.txt'), 'r') as f:
        txt_result = f.readlines()
        crown_start = int(txt_result[0])
        branch_start = int(txt_result[1])
        aorta_end = int(txt_result[2])
       
    patient_copys = []
    for t in range(len(set_window_list)):
        patient_copys.append(set_window_list[t](patient_pixels, 300, 600, crown_start, branch_start))
            
    all_path_list = []
    delta = 0
    continue_flag = False
    while(True):
        start_x = max(int(spine_x-img_width*(0.2+delta)), 0)
        end_x = min(int(spine_x+img_width*(0.2+delta)), img_width)
        start_y = max(int(spine_y-img_height*(0.4+delta)), 0)
        end_y = min(int(spine_y+img_height*(0.1+delta)), img_height)
        for t in range(len(patient_copys)):
            patient_copy = patient_copys[t]
            patient_cut = patient_copy[:, start_y:end_y, start_x:end_x]

            cut_height, cut_width = patient_cut.shape[1:3]
            cut_min_side = min(cut_height, cut_width)
            pre_contours, _ = cv2.findContours(patient_cut[0], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            pre_circle = list(map(lambda x: cv2.minEnclosingCircle(x), pre_contours))
            path_dict = {}
            for i in range(1, len(patient_cut)):
                cur_contours, _ = cv2.findContours(patient_cut[i], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                cur_circle = list(map(lambda x: cv2.minEnclosingCircle(x), cur_contours))
                for j in range(len(pre_circle)):
                    pre_out_of_range = False
                    if pre_circle[j][0][0] < (0.4-delta)*cut_width \
                    or pre_circle[j][0][0] < (0.6-delta)*cut_width and pre_circle[j][0][1] > (0.8+delta)*cut_height:
                        pre_out_of_range = True
                    candidate_list = []
                    for k in range(len(cur_circle)):
                        cur_out_of_range = False
                        if cur_circle[k][0][0] < (0.4-delta)*cut_width \
                        or cur_circle[k][0][0] < (0.6-delta)*cut_width and cur_circle[k][0][1] > (0.8+delta)*cut_height:
                            cur_out_of_range = True
                        dis = sqrt((pre_circle[j][0][0]-cur_circle[k][0][0])**2+(pre_circle[j][0][1]-cur_circle[k][0][1])**2)
                        max_r, min_r = max(pre_circle[j][1], cur_circle[k][1]), min(pre_circle[j][1], cur_circle[k][1])
                        if dis <= max_r:
                            #candidate_list.append((k, cv2.contourArea(cur_contours[k])))
                            candidate_list.append((k, cur_out_of_range))

                    if len(candidate_list) == 0:
                        continue
                    #candidate_list.sort(key=lambda x:x[1])
                    path = path_dict.get((i-1, j), Elem((i-1, j), pre_contours[j], t, pre_out_of_range))
                    temp_path_list = [copy.deepcopy(path) for s in range(len(candidate_list))]
                    put_back_flag = False
                    for s, candidate in enumerate(candidate_list):
                        k = candidate[0]
                        cur_out_of_range = candidate[1]
                        if get_intersection(patient_cut[i], pre_contours[j], cur_contours[k]) < 0.6:
                            continue
                        temp_path_list[s].append((i, k), cur_contours[k], cur_out_of_range)
                        if not (i, k) in path_dict \
                        or path_dict[(i, k)].out_of_range and not temp_path_list[s].out_of_range \
                        or len(path_dict[(i, k)]) < len(temp_path_list[s]) \
                        or path_dict[(i, k)].get_prev_area() < temp_path_list[s].get_prev_area():
                            path_dict[(i, k)] = temp_path_list[s]
                            put_back_flag = True
                    if not put_back_flag:
                        path_dict[(i-1, j)] = path

                pre_contours = cur_contours
                pre_circle = cur_circle

            path_list = list(path_dict.values())
            all_path_list += path_list

        print('all_path_list length:', len(all_path_list))
        if len(all_path_list) == 0:
            print("Can't find aorta!")
            continue_flag = True
            break
        all_path_list.sort(key=lambda x: len(x), reverse=True)
        for i in range(min(3, len(all_path_list))):
            print(all_path_list[i].t, ':', len(all_path_list[i]), all_path_list[i].root, '->', all_path_list[i].end, all_path_list[i].out_of_range)
        path = None
        for tp in all_path_list:
            if tp.out_of_range == False:
                path = tp
                break
        length_factor = 0.6
        if patient_pixels.shape[0] < 300:
            length_factor = 0.45
        if len(path) < length_factor*patient_pixels.shape[0] \
        or path.root[0] > crown_start or path.end[0]+1 < aorta_end:#or all_path_list[0].root[0] > 0.2*patient_pixels.shape[0] \
            delta += 0.05
            all_path_list = []
            if delta > 0.35:
                print("Can't find aorta!")
                continue_flag = True
                break
        else:
            break
                
    if continue_flag:
        continue
    
    start, end = path.root[0], path.end[0]+1
    path.get_bboxes()
    
    branch_start_ori = branch_start
    branch_start += 3
    pre_w = path.bboxes[branch_start-start][2]-path.bboxes[branch_start-start][0]+1
    while branch_start > branch_start_ori-3:
        cur_w = path.bboxes[branch_start-start-1][2]-path.bboxes[branch_start-start-1][0]+1
        if cur_w - pre_w > 0.4 * pre_w:
            break
        branch_start -= 1
        pre_w = cur_w
    if branch_start == branch_start_ori-3:
        branch_start = branch_start_ori
    
    get_branch = True
    try:
        assert branch_start > crown_start
        branch_path_list = []
        for tmp_path in all_path_list:
            if tmp_path.t != path.t or tmp_path.root != path.root or len(tmp_path) < branch_start - start + 1:
                continue
            if tmp_path.keys[branch_start-start-1] == path.keys[branch_start-start-1] and tmp_path.keys[branch_start-start] != path.keys[branch_start-start]:
                branch_path_list.append(tmp_path)
        branch = branch_path_list[0]
        find_branch_flag = False
        for tb in branch_path_list:
            tb.get_bboxes()
            tb_start_w = 0
            for tb_i in range(min(5, len(tb))):
                tb_start_w += tb.bboxes[branch_start-start+tb_i][2]-tb.bboxes[branch_start-start+tb_i][0]+1
            tb_start_w /= tb_i+1
            tb_start_x = tb_pre_x = (tb.bboxes[branch_start-start][0]+tb.bboxes[branch_start-start][2])//2
            tb_pre_y = (tb.bboxes[branch_start-start][1]+tb.bboxes[branch_start-start][3])//2
            tb_pre_w = tb.bboxes[branch_start-start][2]-tb.bboxes[branch_start-start][0]+1
            tb_i = branch_start+1
            break_flag = False
            while tb_i < tb.end[0]+1:
                tb_cur_w = tb.bboxes[tb_i-start][2]-tb.bboxes[tb_i-start][0]+1
                tb_cur_x = (tb.bboxes[tb_i-start][2]+tb.bboxes[tb_i-start][0])/2
                tb_cur_y = (tb.bboxes[tb_i-start][3]+tb.bboxes[tb_i-start][1])/2
                if tb_cur_y > tb_pre_y and tb_cur_x < tb_pre_x and tb_pre_w - tb_cur_w > tb_cur_w * 0.45:
                    break
                if tb_i-branch_start > 1 and tb_cur_x > tb_start_x and abs(tb_cur_w - tb_start_w) > 0.4 * tb_start_w:
                    break_flag = True
                    break
                tb_pre_w, tb_pre_x, tb_pre_y = tb_cur_w, tb_cur_x, tb_cur_y
                tb_i += 1
            if break_flag:
                branch = tb
                branch_end = tb_i
                find_branch_flag = True
                break
        
        if not find_branch_flag:
            branch.get_bboxes()
            branch_end = branch_start+1
            branch_start_w = 0
            for b_i in range(min(5, len(branch))):
                branch_start_w += branch.bboxes[branch_start-start+b_i][2]-branch.bboxes[branch_start-start+b_i][0]+1
            branch_start_w /= b_i+1
            branch_start_x = pre_x = (branch.bboxes[branch_start-start][0]+branch.bboxes[branch_start-start][2])//2
            pre_y = (branch.bboxes[branch_start-start][1]+branch.bboxes[branch_start-start][3])//2
            pre_w = branch.bboxes[branch_start-start][2]-branch.bboxes[branch_start-start][0]+1
            while branch_end < branch.end[0]+1:
                cur_w = branch.bboxes[branch_end-start][2]-branch.bboxes[branch_end-start][0]+1
                cur_x = (branch.bboxes[branch_end-start][2]+branch.bboxes[branch_end-start][0])/2
                cur_y = (branch.bboxes[branch_end-start][3]+branch.bboxes[branch_end-start][1])/2
                if branch_end-branch_start > 1 and (cur_x > branch_start_x and abs(cur_w - branch_start_w) > 0.4 * branch_start_w \
                or cur_y > pre_y and cur_x < pre_x and pre_w - cur_w > cur_w * 0.45):
                    break
                branch_end += 1
                pre_w, pre_x, pre_y = cur_w, cur_x, cur_y
        assert branch_end < end
    except:
        traceback.print_exc()
        branch = all_path_list[-1]
        get_branch = False
        branch_start = branch_end = -1
    print(f'crown_start:{crown_start}, aorta_end:{aorta_end}, branch_start:{branch_start}, branch_end:{branch_end}')
    
    patient = load_scan(input_folder_list[p])
    draw_bbox(patient, path, start_x, start_y, branch, branch_start, branch_end, crown_start, aorta_end, os.path.join(input_folder_list[p], 'save_raw'))
    
    path.blur_bboxes(kind=0)
    if get_branch:
        branch.blur_bboxes(kind=0)
    patient = load_scan(input_folder_list[p])
    draw_bbox(patient, path, start_x, start_y, branch, branch_start, branch_end, crown_start, aorta_end, os.path.join(input_folder_list[p], 'save_raw_blur'))
    
    path.post_process(patient_pixels, start_x, start_y)
    if get_branch:
        branch.post_process(patient_pixels, start_x, start_y)
    patient = load_scan(input_folder_list[p])
    draw_post_bbox(patient, path, branch, branch_start, branch_end, crown_start, aorta_end, os.path.join(input_folder_list[p], 'save_post'))
    
    path.blur_bboxes(kind=1)
    if get_branch:
        branch.blur_bboxes(kind=1)
    patient = load_scan(input_folder_list[p])
    draw_post_bbox(patient, path, branch, branch_start, branch_end, crown_start, aorta_end, os.path.join(input_folder_list[p], 'save_processed'))

******Processing: /nfs3-p1/zsxm/dataset/aorta/chuguangzhou-Im46-111/2/  ******
sequence length: 698, height: 512, width: 512
find spine area: 798.0
times:0.49999999999999994
all_path_list length: 17579
0 : 501 (33, 6) -> (533, 0) True
0 : 500 (33, 6) -> (532, 0) True
0 : 499 (33, 6) -> (531, 1) True
all_path_list length: 24216
2 : 410 (125, 5) -> (534, 5) True
2 : 409 (125, 5) -> (533, 4) True
2 : 408 (125, 5) -> (532, 7) True
all_path_list length: 30235
2 : 535 (0, 10) -> (534, 10) True
2 : 534 (0, 10) -> (533, 7) True
2 : 533 (0, 10) -> (532, 10) True
crown_start:94, aorta_end:340, branch_start:113, branch_end:177


In [ ]:
def draw_bbox(slices, path, start_x, start_y, branch, branch_start, branch_end, crown_start, aorta_end, save_path):
    sl = save_path.split('/')
    name = sl[-3] if sl[-1] != '' else sl[-4]
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    if not os.path.exists(os.path.join(save_path, 'picture')):
        os.mkdir(os.path.join(save_path, 'picture'))
    start = path.root[0]
    img_height, img_width = slices[0].pixel_array.shape
    for i, s in enumerate(slices):
        img = s.pixel_array
        intercept = slices[i].RescaleIntercept
        slope = slices[i].RescaleSlope
        if crown_start <= i < aorta_end:
            x1, y1 = path.bboxes[i-start][0] + start_x, path.bboxes[i-start][1] + start_y
            x2, y2 = path.bboxes[i-start][2] + start_x, path.bboxes[i-start][3] + start_y
            if x1>=0 and y1>=0 and x2<=img_width and y2<=img_height:
                picture = img[y1:y2+1, x1:x2+1].astype(np.int16)
                if slope != 1:
                    picture = (slope * picture.astype(np.float64)).astype(np.int16)
                picture += np.int16(intercept)
                picture = np.clip(picture, -100, 500)
                picture = ((picture+100)/600*255).astype(np.uint8)
                picture = Image.fromarray(picture)
                picture.save(os.path.join(save_path, f'picture/1_{name}_{i}.png'))
        if branch_start <= i < branch_end:
            x1, y1 = branch.bboxes[i-start][0] + start_x, branch.bboxes[i-start][1] + start_y
            x2, y2 = branch.bboxes[i-start][2] + start_x, branch.bboxes[i-start][3] + start_y
            if x1>=0 and y1>=0 and x2<=img_width and y2<=img_height:
                picture = img[y1:y2+1, x1:x2+1].astype(np.int16)
                if slope != 1:
                    picture = (slope * picture.astype(np.float64)).astype(np.int16)
                picture += np.int16(intercept)
                picture = np.clip(picture, -100, 500)
                picture = ((picture+100)/600*255).astype(np.uint8)
                picture = Image.fromarray(picture)
                picture.save(os.path.join(save_path, f'picture/2_{name}_{i}.png'))
        if path.root[0] <= i <= path.end[0]:
            x1, y1 = path.bboxes[i-start][0] + start_x, path.bboxes[i-start][1] + start_y
            x2, y2 = path.bboxes[i-start][2] + start_x, path.bboxes[i-start][3] + start_y
            cv2.rectangle(img, (x1, y1), (x2, y2), 3000, 1)
            if branch_start <= i < branch_end:
                x1, y1 = branch.bboxes[i-start][0] + start_x, branch.bboxes[i-start][1] + start_y
                x2, y2 = branch.bboxes[i-start][2] + start_x, branch.bboxes[i-start][3] + start_y
                cv2.rectangle(img, (x1, y1), (x2, y2), 3000, 1)
        s.PixelData = pydicom.encaps.encapsulate([img.tobytes()])
        s.file_meta.TransferSyntaxUID = '1.2.840.10008.1.2.1'
        s.save_as(os.path.join(save_path, f'{i}.dcm'), write_like_original=False)
        
def draw_post_bbox(slices, path, branch, branch_start, branch_end, crown_start, aorta_end, save_path):
    sl = save_path.split('/')
    name = sl[-3] if sl[-1] != '' else sl[-4]
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    if not os.path.exists(os.path.join(save_path, 'picture')):
        os.mkdir(os.path.join(save_path, 'picture'))
    start = path.root[0]
    img_height, img_width = slices[0].pixel_array.shape
    for i, s in enumerate(slices):
        img = s.pixel_array
        intercept = slices[i].RescaleIntercept
        slope = slices[i].RescaleSlope
        if crown_start <= i < aorta_end:
            x1, y1 = path.post_bboxes[i-start][0], path.post_bboxes[i-start][1]
            x2, y2 = path.post_bboxes[i-start][2], path.post_bboxes[i-start][3]
            if x1>=0 and y1>=0 and x2<=img_width and y2<=img_height:
                picture = img[y1:y2+1, x1:x2+1].astype(np.int16)
                if slope != 1:
                    picture = (slope * picture.astype(np.float64)).astype(np.int16)
                picture += np.int16(intercept)
                picture = np.clip(picture, -100, 500)
                picture = ((picture+100)/600*255).astype(np.uint8)
                picture = Image.fromarray(picture)
                picture.save(os.path.join(save_path, f'picture/1_{name}_{i}.png'))
        if branch_start <= i < branch_end:
            x1, y1 = branch.post_bboxes[i-start][0], branch.post_bboxes[i-start][1]
            x2, y2 = branch.post_bboxes[i-start][2], branch.post_bboxes[i-start][3]
            if x1>=0 and y1>=0 and x2<=img_width and y2<=img_height:
                picture = img[y1:y2+1, x1:x2+1].astype(np.int16)
                if slope != 1:
                    picture = (slope * picture.astype(np.float64)).astype(np.int16)
                picture += np.int16(intercept)
                picture = np.clip(picture, -100, 500)
                picture = ((picture+100)/600*255).astype(np.uint8)
                picture = Image.fromarray(picture)
                picture.save(os.path.join(save_path, f'picture/2_{name}_{i}.png'))
        if path.root[0] <= i <= path.end[0]:
            x1, y1 = path.post_bboxes[i-start][0], path.post_bboxes[i-start][1]
            x2, y2 = path.post_bboxes[i-start][2], path.post_bboxes[i-start][3]
            cv2.rectangle(img, (x1, y1), (x2, y2), 3000, 1)
            if branch_start <= i < branch_end:
                x1, y1 = branch.post_bboxes[i-start][0], branch.post_bboxes[i-start][1]
                x2, y2 = branch.post_bboxes[i-start][2], branch.post_bboxes[i-start][3]
                cv2.rectangle(img, (x1, y1), (x2, y2), 3000, 1)
        s.PixelData = pydicom.encaps.encapsulate([img.tobytes()])
        s.file_meta.TransferSyntaxUID = '1.2.840.10008.1.2.1'
        s.save_as(os.path.join(save_path, f'{i}.dcm'), write_like_original=False)

In [ ]:
******Processing: /nfs3-p1/zsxm/dataset/aorta/chuguangzhou-Im46-111/2/  ******
sequence length: 698, height: 512, width: 512
find spine area: 798.0
times:0.49999999999999994
all_path_list length: 8588
2 : 223 (475, 0) -> (697, 2)
2 : 222 (475, 0) -> (696, 1)
2 : 221 (475, 0) -> (695, 1)
all_path_list length: 14169
2 : 261 (25, 13) -> (285, 18)
2 : 260 (25, 13) -> (284, 22)
2 : 259 (25, 13) -> (283, 23)
all_path_list length: 19922
0 : 490 (44, 19) -> (533, 3)
0 : 489 (44, 19) -> (532, 4)
0 : 488 (44, 19) -> (531, 4)
crown_start:94, aorta_end:340, branch_start:96, branch_end:98
******Processing: /nfs3-p1/zsxm/dataset/aorta/hujiaguo-Im34-58/2/  ******
sequence length: 666, height: 512, width: 512
find spine area: 573.5
times:0.39999999999999997
all_path_list length: 15305
2 : 594 (57, 8) -> (650, 11)
2 : 593 (57, 8) -> (649, 12)
2 : 592 (57, 8) -> (648, 15)
crown_start:74, aorta_end:280, branch_start:97, branch_end:183
******Processing: /nfs3-p1/zsxm/dataset/aorta/modehe-Im34-89/2/  ******
sequence length: 301, height: 512, width: 512
find spine area: 2515.5
times:0.44999999999999996
all_path_list length: 7827
2 : 265 (36, 19) -> (300, 14)
0 : 264 (37, 18) -> (300, 15)
1 : 264 (37, 19) -> (300, 13)
crown_start:60, aorta_end:270, branch_start:90, branch_end:91
******Processing: /nfs3-p1/zsxm/dataset/aorta/wanglianlian-Im42-139/2/  ******
sequence length: 690, height: 512, width: 512
find spine area: 3090.5
times:0.15000000000000002
all_path_list length: 12074
0 : 299 (272, 10) -> (570, 5)
2 : 299 (273, 4) -> (571, 9)
0 : 298 (272, 10) -> (569, 4)
all_path_list length: 16301
2 : 622 (68, 18) -> (689, 20)
2 : 621 (68, 18) -> (688, 15)
2 : 620 (68, 18) -> (687, 15)
crown_start:123, aorta_end:444, branch_start:135, branch_end:219
******Processing: /nfs3-p1/zsxm/dataset/aorta/yuanenchun-Im27-93/2/  ******
sequence length: 279, height: 512, width: 512
find spine area: 1754.0
times:0.2
all_path_list length: 7018
2 : 189 (26, 12) -> (214, 2)
2 : 188 (26, 12) -> (213, 8)
2 : 187 (26, 12) -> (212, 6)
crown_start:37, aorta_end:180, branch_start:42, branch_end:57
******Processing: /nfs3-p1/zsxm/dataset/aorta/yuliyao-Im25-31/2/  ******
sequence length: 672, height: 512, width: 512
find spine area: 993.0
times:0.39999999999999997
all_path_list length: 16236
2 : 451 (35, 7) -> (485, 10)
2 : 450 (35, 7) -> (484, 11)
0 : 449 (36, 6) -> (484, 12)
crown_start:73, aorta_end:93, branch_start:91, branch_end:119
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhangxiyuan-Im9-50/2/  ******
sequence length: 221, height: 512, width: 512
find spine area: 4452.0
times:0.10000000000000002
all_path_list length: 3014
2 : 155 (10, 6) -> (164, 13)
2 : 154 (10, 6) -> (163, 11)
2 : 153 (10, 6) -> (162, 12)
crown_start:22, aorta_end:100, branch_start:30, branch_end:49
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/zhaoqifeng/img/  ******
sequence length: 651, height: 512, width: 512
find spine area: 600.0
times:0.5499999999999999
all_path_list length: 12206
2 : 583 (68, 10) -> (650, 6)
0 : 582 (69, 11) -> (650, 6)
2 : 582 (68, 10) -> (649, 8)
crown_start:86, aorta_end:502, branch_start:114, branch_end:156
******Processing: /nfs3-p1/zsxm/dataset/aortic_dissection/CTA3(+)/+C/  ******
sequence length: 707, height: 512, width: 512
find spine area: 621.5
times:0.5499999999999999
all_path_list length: 10680
2 : 707 (0, 13) -> (706, 9)
2 : 706 (0, 13) -> (705, 6)
2 : 705 (0, 13) -> (704, 10)
crown_start:31, aorta_end:495, branch_start:49, branch_end:105

In [ ]:
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/chenxiu/img/  ******
sequence length: 737, height: 512, width: 512
find spine area: 783.5
times:0.3
all_path_list length: 4414
2 : 737 (0, 15) -> (736, 10)
0 : 687 (0, 6) -> (686, 5)
1 : 687 (0, 3) -> (686, 2)
crown_start:53, aorta_end:566, branch_start:82, branch_end:185
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/dingxing/img/  ******
sequence length: 521, height: 512, width: 512
find spine area: 936.0
times:0.49999999999999994
all_path_list length: 2598
0 : 465 (56, 12) -> (520, 15)
1 : 438 (83, 6) -> (520, 15)
1 : 378 (83, 6) -> (460, 15)
crown_start:141, aorta_end:520, branch_start:167, branch_end:269
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/huanggaoqiang/img/  ******
sequence length: 532, height: 512, width: 512
find spine area: 649.5
times:0.5499999999999999
all_path_list length: 2753
2 : 504 (28, 8) -> (531, 7)
2 : 504 (28, 8) -> (531, 8)
0 : 467 (65, 13) -> (531, 5)
crown_start:81, aorta_end:510, branch_start:105, branch_end:108
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/louhangqing/img/  ******
sequence length: 984, height: 512, width: 512
find spine area: 680.5
times:0.5499999999999999
all_path_list length: 5515
2 : 804 (62, 11) -> (865, 20)
0 : 802 (58, 5) -> (859, 5)
2 : 786 (62, 11) -> (847, 15)
crown_start:103, aorta_end:615, branch_start:128, branch_end:148
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/wangxiaowei/yes/  ******
sequence length: 841, height: 512, width: 512
find spine area: 2160.0
times:0.49999999999999994
all_path_list length: 6698
0 : 791 (50, 11) -> (840, 8)
1 : 781 (60, 12) -> (840, 5)
2 : 774 (67, 21) -> (840, 11)
crown_start:110, aorta_end:648, branch_start:136, branch_end:154
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/xiangxijiao/img/  ******
sequence length: 719, height: 512, width: 512
find spine area: 803.0
times:0.35
all_path_list length: 3769
2 : 555 (0, 11) -> (554, 17)
2 : 528 (0, 11) -> (527, 9)
0 : 521 (0, 13) -> (520, 9)
crown_start:52, aorta_end:445, branch_start:66, branch_end:70
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/yangxiangqin/yes/  ******
sequence length: 808, height: 512, width: 512
find spine area: 2117.5
times:0.15000000000000002
all_path_list length: 4098
0 : 679 (0, 3) -> (678, 5)
1 : 645 (0, 1) -> (644, 0)
0 : 553 (0, 3) -> (552, 7)
crown_start:43, aorta_end:483, branch_start:62, branch_end:179
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/zhaoqifeng/img/  ******
sequence length: 651, height: 512, width: 512
find spine area: 600.0
times:0.5499999999999999
all_path_list length: 3204
2 : 583 (68, 10) -> (650, 6)
0 : 582 (69, 11) -> (650, 6)
0 : 535 (69, 11) -> (603, 11)
crown_start:86, aorta_end:502, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-14-24d075117625>", line 163, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/zhujingchun/img/  ******
sequence length: 807, height: 512, width: 512
find spine area: 676.5
times:0.35
all_path_list length: 5179
0 : 807 (0, 10) -> (806, 11)
2 : 807 (0, 15) -> (806, 9)
2 : 750 (0, 15) -> (749, 20)
crown_start:82, aorta_end:532, branch_start:115, branch_end:140
******Processing: /nfs3-p1/zsxm/dataset/aorta_CTA/zuochun/img/  ******
sequence length: 961, height: 512, width: 512
find spine area: 1128.5
times:0.25
all_path_list length: 4740
2 : 683 (50, 9) -> (732, 10)
2 : 662 (50, 9) -> (711, 13)
2 : 642 (50, 9) -> (691, 12)
crown_start:99, aorta_end:607, branch_start:115, branch_end:168
******Processing: /nfs3-p1/zsxm/dataset/aortic_dissection/CTA1(+)/+C/  ******
sequence length: 513, height: 512, width: 512
find spine area: 672.0
times:0.49999999999999994
all_path_list length: 3212
0 : 456 (0, 7) -> (455, 5)
2 : 456 (0, 14) -> (455, 17)
2 : 431 (0, 14) -> (430, 11)
crown_start:61, aorta_end:266, branch_start:74, branch_end:140
******Processing: /nfs3-p1/zsxm/dataset/aortic_dissection/CTA2(+)/+C/  ******
sequence length: 642, height: 512, width: 512
find spine area: 795.0
times:0.44999999999999996
all_path_list length: 4250
0 : 597 (45, 11) -> (641, 6)
2 : 468 (40, 14) -> (507, 6)
0 : 463 (45, 11) -> (507, 9)
crown_start:116, aorta_end:467, branch_start:152, branch_end:249
******Processing: /nfs3-p1/zsxm/dataset/aortic_dissection/CTA2(-)/+c/  ******
sequence length: 807, height: 512, width: 512
find spine area: 676.5
times:0.35
all_path_list length: 5179
0 : 807 (0, 10) -> (806, 11)
2 : 807 (0, 15) -> (806, 9)
2 : 750 (0, 15) -> (749, 20)
crown_start:80, aorta_end:573, branch_start:115, branch_end:140
******Processing: /nfs3-p1/zsxm/dataset/aortic_dissection/CTA3(+)/+C/  ******
sequence length: 707, height: 512, width: 512
find spine area: 621.5
times:0.5499999999999999
all_path_list length: 3551
2 : 707 (0, 13) -> (706, 9)
2 : 570 (0, 13) -> (569, 12)
2 : 284 (0, 13) -> (283, 6)
crown_start:31, aorta_end:495, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-14-24d075117625>", line 163, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range


******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/zhoujun-Im33-128/2/  ******
sequence length: 253, height: 512, width: 512
find spine area: 560.5
times:0.5499999999999999
all_path_list length: 2155
2 : 216 (28, 11) -> (243, 11)
2 : 214 (28, 11) -> (241, 10)
0 : 166 (31, 9) -> (196, 7)
crown_start:35, aorta_end:133, branch_start:41, branch_end:50
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/zhouzheyu-Im32-152/2/  ******
sequence length: 703, height: 512, width: 512
find spine area: 598.0
times:0.25
all_path_list length: 5384
2 : 535 (5, 14) -> (539, 19)
2 : 535 (5, 14) -> (539, 17)
2 : 531 (5, 14) -> (535, 10)
crown_start:88, aorta_end:440, branch_start:112, branch_end:113
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/zhudehong-Im22-91/2/  ******
sequence length: 650, height: 512, width: 512
find spine area: 2422.0
times:0.44999999999999996
all_path_list length: 3076
0 : 530 (82, 8) -> (611, 6)
2 : 504 (120, 6) -> (623, 6)
2 : 498 (120, 6) -> (617, 10)
crown_start:98, aorta_end:435, branch_start:108, branch_end:130
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/zhuyongbu-im23-92/2/  ******
sequence length: 308, height: 666, width: 512
find spine area: 4623.0
times:0.6
all_path_list length: 1770
2 : 96 (141, 16) -> (236, 18)
2 : 96 (212, 2) -> (307, 2)
2 : 93 (141, 16) -> (233, 19)
all_path_list length: 3062
2 : 245 (11, 20) -> (255, 7)
2 : 243 (11, 20) -> (253, 14)
0 : 230 (24, 26) -> (253, 13)
crown_start:37, aorta_end:210, branch_start:51, branch_end:89
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/chuguangzhou-Im46-111/2/  ******
sequence length: 698, height: 512, width: 512
find spine area: 798.0
times:0.49999999999999994
all_path_list length: 1981
2 : 223 (475, 0) -> (697, 2)
2 : 207 (79, 9) -> (285, 13)
2 : 203 (79, 9) -> (281, 17)
all_path_list length: 3072
2 : 261 (25, 13) -> (285, 18)
2 : 257 (25, 13) -> (281, 22)
2 : 240 (25, 13) -> (264, 24)
all_path_list length: 4496
0 : 490 (44, 19) -> (533, 3)
0 : 473 (44, 19) -> (516, 7)
1 : 444 (79, 3) -> (522, 9)
crown_start:94, aorta_end:340, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-9-7ed4ab80bf91>", line 162, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/dongxuesheng-Im37-149/2/  ******
sequence length: 684, height: 512, width: 512
find spine area: 3695.0
times:0.15000000000000002
all_path_list length: 3343
2 : 684 (0, 14) -> (683, 16)
2 : 670 (0, 14) -> (669, 7)
0 : 613 (44, 9) -> (656, 7)
crown_start:80, aorta_end:441, branch_start:96, branch_end:136
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/durenxi-Im57-172/2/  ******
sequence length: 723, height: 512, width: 512
find spine area: 5924.5
times:0.15000000000000002
all_path_list length: 2730
0 : 485 (88, 5) -> (572, 2)
1 : 484 (88, 4) -> (571, 2)
2 : 475 (100, 10) -> (574, 1)
crown_start:132, aorta_end:480, branch_start:139, branch_end:167
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/fangcaiyu-Im21-91/2/  ******
sequence length: 267, height: 512, width: 512
find spine area: 1856.5
times:0.35
all_path_list length: 2170
2 : 207 (0, 6) -> (206, 10)
2 : 193 (0, 6) -> (192, 14)
0 : 187 (7, 14) -> (193, 8)
crown_start:38, aorta_end:170, branch_start:43, branch_end:64
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/fengyonghua-Im21-89/2/  ******
sequence length: 255, height: 512, width: 512
find spine area: 843.5
times:0.5499999999999999
all_path_list length: 2464
0 : 192 (0, 6) -> (191, 12)
1 : 192 (0, 7) -> (191, 6)
0 : 159 (0, 6) -> (158, 12)
crown_start:32, aorta_end:174, branch_start:38, branch_end:72
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/fuhuajin-Im29-126/2/  ******
sequence length: 666, height: 512, width: 512
find spine area: 2127.0
times:0.44999999999999996
all_path_list length: 4420
0 : 472 (194, 7) -> (665, 2)
0 : 372 (194, 7) -> (565, 8)
2 : 344 (193, 9) -> (536, 7)
all_path_list length: 6210
0 : 486 (180, 13) -> (665, 4)
0 : 463 (180, 13) -> (642, 4)
0 : 386 (180, 13) -> (565, 10)
all_path_list length: 7908
0 : 509 (157, 11) -> (665, 6)
0 : 486 (157, 11) -> (642, 7)
0 : 409 (157, 11) -> (565, 10)
all_path_list length: 9451
0 : 643 (23, 13) -> (665, 9)
0 : 620 (23, 13) -> (642, 9)
0 : 543 (23, 13) -> (565, 10)
crown_start:53, aorta_end:372, branch_start:73, branch_end:162
******Processing: /nfs3-p1/zsxm/dataset/aorta/.complete/gaomingsheng-Im34-151/2/  ******
sequence length: 720, height: 512, width: 512
find spine area: 1020.0
times:0.35
all_path_list length: 4011
2 : 600 (78, 16) -> (677, 7)
0 : 479 (67, 10) -> (545, 16)
0 : 443 (67, 10) -> (509, 11)
crown_start:102, aorta_end:435, branch_start:122, branch_end:1882.0
******Processing: /nfs3-p1/zsxm/dataset/aorta/gengyongchao-Im21-83/2/  ******
sequence length: 122, height: 512, width: 512
find spine area: 1550.0
times:0.6
all_path_list length: 1272
1 : 91 (3, 11) -> (93, 12)
1 : 90 (3, 11) -> (92, 11)
0 : 86 (10, 12) -> (95, 11)
crown_start:9, aorta_end:81, branch_start:16, branch_end:22
******Processing: /nfs3-p1/zsxm/dataset/aorta/hebin-Im41-91/2/  ******
sequence length: 270, height: 512, width: 512
find spine area: 1415.0
times:0.3
all_path_list length: 2178
0 : 190 (33, 14) -> (222, 8)
2 : 150 (44, 10) -> (193, 11)
1 : 144 (45, 12) -> (188, 6)
crown_start:50, aorta_end:172, branch_start:56, branch_end:66
******Processing: /nfs3-p1/zsxm/dataset/aorta/hejianjun-Im31-99/2/  ******
sequence length: 639, height: 512, width: 512
find spine area: 1908.0
times:0.6
all_path_list length: 4626
2 : 578 (0, 12) -> (577, 9)
0 : 577 (0, 9) -> (576, 7)
2 : 532 (0, 12) -> (531, 10)
crown_start:83, aorta_end:319, branch_start:100, branch_end:137
******Processing: /nfs3-p1/zsxm/dataset/aorta/heshengnan-Im16-90/2/  ******
sequence length: 253, height: 512, width: 512
find spine area: 549.0
times:0.49999999999999994
all_path_list length: 2729
2 : 200 (0, 14) -> (199, 8)
2 : 190 (0, 14) -> (189, 15)
1 : 187 (3, 7) -> (189, 9)
crown_start:33, aorta_end:160, branch_start:38, branch_end:69
******Processing: /nfs3-p1/zsxm/dataset/aorta/huangjianhua-Im38-144/2/  ******
sequence length: 648, height: 512, width: 512
find spine area: 2561.0
times:0.25
all_path_list length: 3178
2 : 406 (242, 2) -> (647, 9)
2 : 388 (242, 2) -> (629, 6)
2 : 373 (197, 5) -> (569, 21)
all_path_list length: 4892
0 : 596 (4, 15) -> (599, 9)
0 : 586 (4, 15) -> (589, 19)
0 : 473 (4, 15) -> (476, 9)
crown_start:96, aorta_end:432, branch_start:109, branch_end:189
******Processing: /nfs3-p1/zsxm/dataset/aorta/hujiaguo-Im34-58/2/  ******
sequence length: 666, height: 512, width: 512
find spine area: 573.5
times:0.39999999999999997
all_path_list length: 3415
2 : 594 (57, 8) -> (650, 11)
2 : 585 (57, 8) -> (641, 9)
0 : 508 (57, 4) -> (564, 8)
crown_start:74, aorta_end:280, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-6-7ed4ab80bf91>", line 162, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range
******Processing: /nfs3-p1/zsxm/dataset/aorta/jiangquanfeng-Im 50-107/2/  ******
sequence length: 537, height: 512, width: 512
find spine area: 1497.0
times:0.6
all_path_list length: 2321
0 : 383 (0, 8) -> (382, 8)
1 : 383 (0, 4) -> (382, 9)
2 : 383 (0, 8) -> (382, 11)
crown_start:64, aorta_end:293, branch_start:82, branch_end:87
******Processing: /nfs3-p1/zsxm/dataset/aorta/litingxian-Im36-141/2/  ******
sequence length: 618, height: 512, width: 512
find spine area: 804.5
times:0.6
all_path_list length: 2862
2 : 271 (347, 8) -> (617, 11)
2 : 264 (347, 8) -> (610, 5)
2 : 208 (353, 11) -> (560, 12)
all_path_list length: 4232
2 : 359 (259, 11) -> (617, 11)
2 : 359 (259, 11) -> (617, 14)
0 : 354 (129, 2) -> (482, 5)
all_path_list length: 6245
2 : 571 (47, 24) -> (617, 21)
2 : 571 (47, 24) -> (617, 26)
0 : 498 (29, 13) -> (526, 2)
crown_start:75, aorta_end:420, branch_start:92, branch_end:115
******Processing: /nfs3-p1/zsxm/dataset/aorta/luohuafa-im29-96/2/  ******
sequence length: 323, height: 666, width: 512
find spine area: 1336.5
times:0.25
all_path_list length: 2004
2 : 275 (0, 16) -> (274, 11)
0 : 236 (4, 10) -> (239, 6)
0 : 233 (4, 10) -> (236, 8)
crown_start:38, aorta_end:230, branch_start:47, branch_end:80
******Processing: /nfs3-p1/zsxm/dataset/aorta/lushida-Im33-159/2/  ******
sequence length: 705, height: 512, width: 512
find spine area: 712.5
times:0.39999999999999997
all_path_list length: 3943
2 : 383 (105, 2) -> (487, 14)
0 : 369 (119, 4) -> (487, 6)
1 : 365 (123, 1) -> (487, 5)
all_path_list length: 5321
2 : 600 (105, 3) -> (704, 22)
2 : 592 (105, 3) -> (696, 16)
2 : 570 (105, 3) -> (674, 12)
all_path_list length: 6306
2 : 600 (105, 4) -> (704, 24)
2 : 592 (105, 4) -> (696, 18)
2 : 570 (105, 4) -> (674, 13)
all_path_list length: 7216
2 : 600 (105, 4) -> (704, 23)
2 : 592 (105, 4) -> (696, 18)
2 : 587 (105, 4) -> (691, 14)
all_path_list length: 8393
2 : 600 (105, 4) -> (704, 24)
2 : 592 (105, 4) -> (696, 20)
2 : 587 (105, 4) -> (691, 17)
all_path_list length: 8491
2 : 600 (105, 4) -> (704, 24)
2 : 592 (105, 4) -> (696, 20)
2 : 587 (105, 4) -> (691, 16)
all_path_list length: 8496
2 : 600 (105, 4) -> (704, 24)
2 : 592 (105, 4) -> (696, 20)
2 : 587 (105, 4) -> (691, 16)
all_path_list length: 8496
2 : 600 (105, 4) -> (704, 24)
2 : 592 (105, 4) -> (696, 20)
2 : 587 (105, 4) -> (691, 16)
Can't find aorta!
******Processing: /nfs3-p1/zsxm/dataset/aorta/lvpeiji-Im32-136/2/  ******
sequence length: 636, height: 512, width: 512
find spine area: 786.0
times:0.44999999999999996
all_path_list length: 2866
2 : 431 (0, 11) -> (430, 11)
0 : 430 (0, 8) -> (429, 7)
1 : 399 (30, 6) -> (428, 4)
crown_start:86, aorta_end:425, branch_start:108, branch_end:123
******Processing: /nfs3-p1/zsxm/dataset/aorta/manyanfeng-Im32-153/2/  ******
sequence length: 660, height: 512, width: 512
find spine area: 620.0
times:0.25
all_path_list length: 4382
2 : 660 (0, 10) -> (659, 20)
2 : 633 (0, 10) -> (632, 9)
2 : 528 (0, 10) -> (527, 8)
crown_start:91, aorta_end:355, branch_start:111, branch_end:117
******Processing: /nfs3-p1/zsxm/dataset/aorta/maoliang-Im12-90/2/  ******
sequence length: 253, height: 512, width: 512
find spine area: 822.5
times:0.2
all_path_list length: 2140
2 : 196 (5, 13) -> (200, 8)
0 : 184 (17, 6) -> (200, 6)
2 : 173 (5, 13) -> (177, 11)
crown_start:18, aorta_end:163, branch_start:33, branch_end:44
******Processing: /nfs3-p1/zsxm/dataset/aorta/modehe-Im34-89/2/  ******
sequence length: 301, height: 512, width: 512
find spine area: 2515.5
times:0.44999999999999996
all_path_list length: 2093
2 : 265 (36, 19) -> (300, 14)
0 : 264 (37, 18) -> (300, 15)
1 : 264 (37, 19) -> (300, 13)
crown_start:60, aorta_end:270, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-6-7ed4ab80bf91>", line 162, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range
******Processing: /nfs3-p1/zsxm/dataset/aorta/nibingshan-Im33-42/2/  ******
sequence length: 696, height: 512, width: 512
find spine area: 584.5
times:0.5499999999999999
all_path_list length: 4089
2 : 696 (0, 13) -> (695, 13)
0 : 684 (0, 9) -> (683, 2)
2 : 662 (0, 13) -> (661, 5)
crown_start:90, aorta_end:345, branch_start:98, branch_end:99
******Processing: /nfs3-p1/zsxm/dataset/aorta/qiquanping-Im25-54/2/  ******
sequence length: 1073, height: 512, width: 512
find spine area: 679.5
times:0.5499999999999999
all_path_list length: 6427
0 : 734 (0, 16) -> (733, 13)
2 : 700 (36, 14) -> (735, 7)
0 : 675 (0, 16) -> (674, 12)
crown_start:131, aorta_end:655, branch_start:164, branch_end:180
******Processing: /nfs3-p1/zsxm/dataset/aorta/shengfumei-Im31-132/2/  ******
sequence length: 612, height: 512, width: 512
find spine area: 740.5
times:0.3
all_path_list length: 3518
0 : 595 (12, 10) -> (606, 8)
2 : 592 (20, 10) -> (611, 17)
2 : 571 (20, 10) -> (590, 12)
crown_start:86, aorta_end:390, branch_start:94, branch_end:112
******Processing: /nfs3-p1/zsxm/dataset/aorta/shenhuagou-Im33-95/2/  ******
sequence length: 633, height: 512, width: 512
find spine area: 533.5
times:0.44999999999999996
all_path_list length: 3511
2 : 486 (30, 16) -> (515, 8)
0 : 480 (32, 11) -> (511, 5)
2 : 466 (30, 16) -> (495, 12)
crown_start:85, aorta_end:300, branch_start:98, branch_end:165
******Processing: /nfs3-p1/zsxm/dataset/aorta/shenjiaren-Im35-136/2/  ******
sequence length: 255, height: 512, width: 512
find spine area: 1455.0
times:0.49999999999999994
all_path_list length: 2639
2 : 188 (0, 17) -> (187, 13)
2 : 184 (0, 17) -> (183, 13)
0 : 171 (13, 11) -> (183, 12)
crown_start:29, aorta_end:142, branch_start:35, branch_end:63
******Processing: /nfs3-p1/zsxm/dataset/aorta/shenjie-Im36-100/2/  ******
sequence length: 715, height: 512, width: 512
find spine area: 4348.5
times:0.35
all_path_list length: 2646
2 : 368 (298, 7) -> (665, 4)
2 : 367 (298, 7) -> (664, 4)
2 : 363 (298, 7) -> (660, 6)
all_path_list length: 4251
2 : 632 (83, 9) -> (714, 14)
2 : 596 (83, 9) -> (678, 7)
2 : 583 (83, 9) -> (665, 8)
crown_start:146, aorta_end:480, branch_start:165, branch_end:185
******Processing: /nfs3-p1/zsxm/dataset/aorta/shiaihua-Im28-108/2/  ******
sequence length: 643, height: 512, width: 512
find spine area: 733.0
times:0.35
all_path_list length: 3728
2 : 461 (12, 13) -> (472, 8)
2 : 457 (12, 13) -> (468, 13)
2 : 452 (12, 13) -> (463, 12)
crown_start:72, aorta_end:314, branch_start:83, branch_end:95
******Processing: /nfs3-p1/zsxm/dataset/aorta/shilei-Im21-92/2/  ******
sequence length: 1129, height: 512, width: 512
find spine area: 667.0
times:0.6
all_path_list length: 5165
0 : 1065 (64, 7) -> (1128, 0)
2 : 1065 (64, 18) -> (1128, 13)
2 : 1025 (64, 18) -> (1088, 7)
crown_start:138, aorta_end:721, branch_start:71, branch_end:75
******Processing: /nfs3-p1/zsxm/dataset/aorta/shuzhiming-Im28-80/2/  ******
sequence length: 129, height: 512, width: 512
find spine area: 692.5
find spine area: 2.0
times:0.3
all_path_list length: 1197
0 : 92 (10, 13) -> (101, 12)
0 : 88 (10, 13) -> (97, 10)
1 : 64 (14, 9) -> (77, 11)
crown_start:12, aorta_end:78, branch_start:17, branch_end:18
******Processing: /nfs3-p1/zsxm/dataset/aorta/taofangguo-Im30-142/2/  ******
sequence length: 639, height: 512, width: 512
find spine area: 639.0
times:0.3
all_path_list length: 4077
0 : 484 (47, 16) -> (530, 1)
2 : 481 (47, 20) -> (527, 8)
2 : 463 (48, 26) -> (510, 14)
crown_start:68, aorta_end:412, branch_start:91, branch_end:167
******Processing: /nfs3-p1/zsxm/dataset/aorta/wangaishui-Im75-128/2/  ******
sequence length: 711, height: 512, width: 512
find spine area: 903.0
times:0.3
all_path_list length: 3038
2 : 680 (20, 10) -> (699, 12)
2 : 539 (20, 10) -> (558, 7)
2 : 497 (20, 10) -> (516, 10)
crown_start:140, aorta_end:410, branch_start:149, branch_end:188
******Processing: /nfs3-p1/zsxm/dataset/aorta/wanggangxiang-Im16-90/2/  ******
sequence length: 745, height: 512, width: 512
find spine area: 559.0
times:0.5499999999999999
all_path_list length: 3964
2 : 474 (131, 10) -> (604, 9)
2 : 454 (131, 10) -> (584, 11)
2 : 448 (131, 10) -> (578, 10)
crown_start:168, aorta_end:540, branch_start:184, branch_end:214
******Processing: /nfs3-p1/zsxm/dataset/aorta/wanggaosheng-Im24-57/2/  ******
sequence length: 135, height: 512, width: 512
find spine area: 906.5
times:0.15000000000000002
all_path_list length: 1110
2 : 97 (11, 14) -> (107, 11)
0 : 94 (14, 11) -> (107, 7)
1 : 92 (14, 15) -> (105, 9)
crown_start:16, aorta_end:69, branch_start:25, branch_end:39
******Processing: /nfs3-p1/zsxm/dataset/aorta/wangjianjun-Im30-110/2/  ******
sequence length: 630, height: 512, width: 512
find spine area: 3041.5
times:0.3
all_path_list length: 2395
2 : 598 (0, 13) -> (597, 7)
2 : 502 (0, 13) -> (501, 5)
2 : 490 (0, 13) -> (489, 5)
crown_start:81, aorta_end:331, branch_start:98, branch_end:147
******Processing: /nfs3-p1/zsxm/dataset/aorta/wanglianlian-Im42-139/2/  ******
sequence length: 690, height: 512, width: 512
find spine area: 3090.5
times:0.15000000000000002
all_path_list length: 2516
0 : 299 (272, 10) -> (570, 5)
2 : 299 (273, 4) -> (571, 9)
2 : 257 (273, 4) -> (529, 3)
all_path_list length: 3766
2 : 622 (68, 18) -> (689, 20)
2 : 559 (68, 18) -> (626, 9)
0 : 542 (69, 6) -> (610, 7)
crown_start:123, aorta_end:444, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-6-7ed4ab80bf91>", line 162, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range
******Processing: /nfs3-p1/zsxm/dataset/aorta/wangmanying-Im23-134/2/  ******
sequence length: 588, height: 512, width: 512
find spine area: 2113.5
find spine area: 590.0
times:0.35
all_path_list length: 3249
2 : 428 (0, 7) -> (427, 8)
2 : 412 (0, 7) -> (411, 13)
2 : 409 (0, 7) -> (408, 11)
crown_start:53, aorta_end:383, branch_start:64, branch_end:167
******Processing: /nfs3-p1/zsxm/dataset/aorta/wangzhongwen-Im36-80/2/  ******
sequence length: 133, height: 512, width: 512
find spine area: 4727.0
times:0.35
all_path_list length: 1100
2 : 102 (0, 7) -> (101, 11)
1 : 91 (15, 9) -> (105, 5)
0 : 90 (15, 8) -> (104, 8)
crown_start:21, aorta_end:82, branch_start:25, branch_end:27
******Processing: /nfs3-p1/zsxm/dataset/aorta/wuguangde-im55-205/2/  ******
sequence length: 299, height: 512, width: 512
find spine area: 2193.5
find spine area: 251.5
find spine area: 156.5
times:0.35
all_path_list length: 2164
0 : 202 (59, 14) -> (260, 11)
2 : 201 (61, 16) -> (261, 10)
2 : 176 (61, 16) -> (236, 14)
crown_start:73, aorta_end:200, branch_start:79, branch_end:111
******Processing: /nfs3-p1/zsxm/dataset/aorta/xiewenbo-Im25-40/2/  ******
sequence length: 1033, height: 512, width: 512
find spine area: 532.5
times:0.6
all_path_list length: 5008
2 : 958 (0, 11) -> (957, 7)
0 : 921 (14, 7) -> (934, 6)
1 : 862 (26, 4) -> (887, 1)
crown_start:134, aorta_end:428, branch_start:165, branch_end:245
******Processing: /nfs3-p1/zsxm/dataset/aorta/xujinbiao-Im37-146/2/  ******
sequence length: 651, height: 512, width: 512
find spine area: 804.5
times:0.35
all_path_list length: 4019
2 : 566 (75, 12) -> (640, 8)
2 : 438 (75, 12) -> (512, 12)
0 : 410 (63, 12) -> (472, 10)
crown_start:87, aorta_end:418, branch_start:110, branch_end:120
******Processing: /nfs3-p1/zsxm/dataset/aorta/xujinyang-Im15-30/2/  ******
sequence length: 139, height: 512, width: 512
find spine area: 1226.5
times:0.35
all_path_list length: 1297
0 : 94 (9, 10) -> (102, 8)
2 : 93 (9, 10) -> (101, 10)
0 : 92 (9, 10) -> (100, 11)
crown_start:14, aorta_end:18, branch_start:18, branch_end:37
******Processing: /nfs3-p1/zsxm/dataset/aorta/yangwenbin-Im44-125/2/  ******
sequence length: 699, height: 512, width: 512
find spine area: 537.0
times:0.3
all_path_list length: 2818
2 : 339 (92, 18) -> (430, 13)
2 : 316 (92, 18) -> (407, 14)
2 : 312 (92, 18) -> (403, 20)
all_path_list length: 4112
2 : 339 (92, 24) -> (430, 18)
2 : 316 (92, 24) -> (407, 18)
2 : 309 (92, 24) -> (400, 27)
all_path_list length: 5488
2 : 517 (0, 12) -> (516, 17)
2 : 485 (0, 12) -> (484, 22)
2 : 473 (0, 12) -> (472, 16)
crown_start:125, aorta_end:388, branch_start:141, branch_end:206
******Processing: /nfs3-p1/zsxm/dataset/aorta/yangyoufu-Im16-25/2/  ******
sequence length: 122, height: 512, width: 512
find spine area: 1817.0
times:0.3
all_path_list length: 1498
0 : 98 (0, 12) -> (97, 3)
2 : 92 (7, 17) -> (98, 11)
2 : 89 (7, 17) -> (95, 14)
crown_start:15, aorta_end:20, branch_start:19, branch_end:37
******Processing: /nfs3-p1/zsxm/dataset/aorta/yaoliang-Im30-83/2/  ******
sequence length: 289, height: 512, width: 512
find spine area: 3250.0
times:0.05000000000000002
all_path_list length: 2161
2 : 79 (101, 5) -> (179, 10)
2 : 73 (140, 20) -> (212, 17)
0 : 69 (141, 23) -> (209, 13)
all_path_list length: 2847
2 : 120 (60, 21) -> (179, 10)
2 : 102 (60, 21) -> (161, 8)
2 : 99 (190, 3) -> (288, 8)
all_path_list length: 3567
0 : 183 (25, 19) -> (207, 7)
2 : 177 (31, 12) -> (207, 6)
1 : 159 (31, 10) -> (189, 8)
crown_start:35, aorta_end:176, branch_start:43, branch_end:53
******Processing: /nfs3-p1/zsxm/dataset/aorta/yinghao-Im18-86/2/  ******
sequence length: 215, height: 666, width: 512
find spine area: 1762.5
times:0.3
all_path_list length: 2320
2 : 171 (0, 11) -> (170, 15)
2 : 158 (0, 11) -> (157, 13)
2 : 119 (0, 11) -> (118, 13)
crown_start:28, aorta_end:145, branch_start:36, branch_end:58
******Processing: /nfs3-p1/zsxm/dataset/aorta/yingzhemin-Im58-108/2/  ******
sequence length: 277, height: 512, width: 512
find spine area: 1606.0
times:0.2
all_path_list length: 2214
0 : 196 (11, 12) -> (206, 11)
0 : 196 (11, 12) -> (206, 8)
1 : 195 (11, 12) -> (205, 5)
crown_start:23, aorta_end:177, branch_start:32, branch_end:43
******Processing: /nfs3-p1/zsxm/dataset/aorta/yuanenchun-Im27-93/2/  ******
sequence length: 279, height: 512, width: 512
find spine area: 1754.0
times:0.2
all_path_list length: 2688
2 : 189 (26, 12) -> (214, 2)
2 : 175 (26, 12) -> (200, 13)
1 : 173 (29, 9) -> (201, 18)
crown_start:37, aorta_end:180, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-6-7ed4ab80bf91>", line 162, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range
******Processing: /nfs3-p1/zsxm/dataset/aorta/yujinmin-Im25-78/2/  ******
sequence length: 977, height: 512, width: 512
find spine area: 1882.0
times:0.3
all_path_list length: 4098
0 : 783 (15, 5) -> (797, 2)
2 : 753 (47, 16) -> (799, 8)
0 : 738 (15, 5) -> (752, 6)
crown_start:81, aorta_end:588, branch_start:104, branch_end:234
******Processing: /nfs3-p1/zsxm/dataset/aorta/yukongban-Im30-139/2/  ******
sequence length: 636, height: 512, width: 512
find spine area: 782.0
times:0.49999999999999994
all_path_list length: 4520
2 : 530 (20, 14) -> (549, 15)
0 : 527 (22, 13) -> (548, 13)
1 : 517 (31, 6) -> (547, 7)
crown_start:84, aorta_end:406, branch_start:100, branch_end:159
******Processing: /nfs3-p1/zsxm/dataset/aorta/yuliyao-Im25-31/2/  ******
sequence length: 672, height: 512, width: 512
find spine area: 993.0
times:0.39999999999999997
all_path_list length: 4150
2 : 451 (35, 7) -> (485, 10)
0 : 449 (36, 6) -> (484, 12)
1 : 448 (36, 7) -> (483, 7)
crown_start:73, aorta_end:93, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-6-7ed4ab80bf91>", line 162, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range
******Processing: /nfs3-p1/zsxm/dataset/aorta/zengzhengyou-Im54-145/2/  ******
sequence length: 663, height: 512, width: 512
find spine area: 2725.5
times:0.6
all_path_list length: 3082
0 : 253 (0, 5) -> (252, 7)
0 : 252 (0, 5) -> (251, 11)
0 : 250 (0, 5) -> (249, 8)
all_path_list length: 4579
0 : 494 (0, 11) -> (493, 13)
0 : 490 (0, 11) -> (489, 19)
0 : 486 (0, 11) -> (485, 7)
crown_start:89, aorta_end:428, branch_start:103, branch_end:186
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhangxinliang-Im16-24/2/  ******
sequence length: 58, height: 512, width: 512
find spine area: 1193.0
find spine area: 388.0
times:0.49999999999999994
all_path_list length: 610
0 : 51 (7, 15) -> (57, 10)
1 : 51 (7, 13) -> (57, 12)
2 : 51 (7, 16) -> (57, 12)
crown_start:13, aorta_end:18, branch_start:17, branch_end:34
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhangxiyuan-Im9-50/2/  ******
sequence length: 221, height: 512, width: 512
find spine area: 4452.0
times:0.10000000000000002
all_path_list length: 1437
2 : 155 (10, 6) -> (164, 13)
2 : 152 (10, 6) -> (161, 9)
2 : 146 (10, 6) -> (155, 12)
crown_start:22, aorta_end:100, branch_start:-1, branch_end:-1
Traceback (most recent call last):
  File "<ipython-input-6-7ed4ab80bf91>", line 162, in <module>
    branch = branch_path_list[0]
IndexError: list index out of range
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhaojianbin-Im31-158/2/  ******
sequence length: 669, height: 512, width: 512
find spine area: 2084.5
times:0.35
all_path_list length: 4421
1 : 265 (266, 3) -> (530, 7)
2 : 250 (0, 12) -> (249, 10)
2 : 243 (0, 12) -> (242, 16)
all_path_list length: 5833
2 : 489 (0, 14) -> (488, 12)
2 : 485 (0, 14) -> (484, 17)
2 : 481 (0, 14) -> (480, 15)
crown_start:74, aorta_end:435, branch_start:93, branch_end:232
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhaojun-Im45-103/2/  ******
sequence length: 632, height: 512, width: 512
find spine area: 17894.5
find spine area: 808.0
find spine area: 17.5
find spine area: 5.5
find spine area: 1.0
find spine area: 25370.0
times:0.10000000000000002
all_path_list length: 3112
2 : 227 (0, 11) -> (226, 3)
2 : 220 (326, 10) -> (545, 14)
2 : 219 (326, 10) -> (544, 15)
all_path_list length: 4719
2 : 251 (0, 16) -> (250, 11)
2 : 251 (0, 16) -> (250, 13)
0 : 239 (254, 12) -> (492, 8)
all_path_list length: 6125
2 : 632 (0, 23) -> (631, 30)
2 : 610 (0, 23) -> (609, 22)
0 : 569 (38, 18) -> (606, 16)
crown_start:54, aorta_end:416, branch_start:121, branch_end:196
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhengsiqing-Im35-106/2/  ******
sequence length: 666, height: 512, width: 512
find spine area: 1991.0
times:0.3
all_path_list length: 4321
2 : 371 (31, 14) -> (401, 18)
2 : 355 (31, 14) -> (385, 11)
0 : 341 (45, 17) -> (385, 16)
all_path_list length: 5992
2 : 635 (31, 14) -> (665, 21)
2 : 630 (31, 14) -> (660, 18)
0 : 596 (45, 22) -> (640, 14)
crown_start:99, aorta_end:331, branch_start:114, branch_end:162
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhoufei-Im16-97/2/  ******
sequence length: 581, height: 512, width: 512
find spine area: 1656.0
times:0.49999999999999994
all_path_list length: 5022
0 : 486 (0, 6) -> (485, 8)
0 : 430 (0, 6) -> (429, 12)
0 : 412 (0, 6) -> (411, 10)
crown_start:68, aorta_end:353, branch_start:76, branch_end:133
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhoujinhai-Im14-94/2/  ******
sequence length: 945, height: 512, width: 512
find spine area: 2807.5
times:0.44999999999999996
all_path_list length: 5176
0 : 945 (0, 7) -> (944, 13)
0 : 945 (0, 7) -> (944, 16)
2 : 945 (0, 16) -> (944, 16)
crown_start:78, aorta_end:605, branch_start:107, branch_end:264

In [ ]:
'''
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhoujun-Im33-128/2/  ******
sequence length: 253, height: 512, width: 512
find spine area: 560.5
times:0.5499999999999999
all_path_list length: 1503
0 : 187 (28, 10) -> (214, 7)
1 : 166 (31, 7) -> (196, 7)
2 : 165 (31, 7) -> (195, 5)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhouzheyu-Im32-152/2/  ******
sequence length: 703, height: 512, width: 512
find spine area: 598.0
times:0.25
all_path_list length: 3650
0 : 491 (49, 17) -> (539, 17)
1 : 491 (49, 17) -> (539, 15)
2 : 487 (49, 17) -> (535, 7)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhudehong-Im22-91/2/  ******
sequence length: 650, height: 512, width: 512
find spine area: 2422.0
times:0.44999999999999996
all_path_list length: 2147
0 : 529 (82, 7) -> (610, 8)
1 : 454 (164, 10) -> (617, 7)
2 : 410 (83, 3) -> (492, 5)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhuyongbu-im23-92/2/  ******
sequence length: 308, height: 666, width: 512
find spine area: 4623.0
times:0.6
all_path_list length: 1161
0 : 128 (30, 8) -> (157, 3)
1 : 127 (30, 8) -> (156, 7)
2 : 127 (30, 8) -> (156, 6)
all_path_list length: 2264
0 : 245 (11, 17) -> (255, 6)
1 : 243 (11, 17) -> (253, 10)
2 : 219 (35, 18) -> (253, 11)
******Processing: /nfs3-p1/zsxm/dataset/aorta/chengxuejun-Im17-83/2/  ******
sequence length: 122, height: 512, width: 512
find spine area: 787.5
times:0.6
all_path_list length: 1017
0 : 59 (28, 11) -> (86, 11)
1 : 59 (28, 8) -> (86, 11)
2 : 55 (5, 7) -> (59, 15)
all_path_list length: 1644
0 : 85 (5, 12) -> (89, 13)
1 : 65 (25, 14) -> (89, 9)
2 : 61 (5, 12) -> (65, 7)
******Processing: /nfs3-p1/zsxm/dataset/aorta/chuguangzhou-Im46-111/2/  ******
sequence length: 698, height: 512, width: 512
find spine area: 798.0
times:0.49999999999999994
all_path_list length: 1514
0 : 142 (94, 2) -> (235, 1)
1 : 139 (125, 3) -> (263, 8)
2 : 139 (261, 7) -> (399, 9)
all_path_list length: 2414
0 : 234 (300, 7) -> (533, 2)
1 : 227 (471, 2) -> (697, 3)
2 : 223 (300, 5) -> (522, 6)
all_path_list length: 3814
0 : 490 (44, 12) -> (533, 2)
1 : 464 (44, 12) -> (507, 5)
2 : 444 (79, 3) -> (522, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/dongxuesheng-Im37-149/2/  ******
sequence length: 684, height: 512, width: 512
find spine area: 3695.0
times:0.15000000000000002
all_path_list length: 2341
0 : 452 (0, 13) -> (451, 8)
1 : 408 (44, 8) -> (451, 4)
2 : 304 (44, 8) -> (347, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/durenxi-Im57-172/2/  ******
sequence length: 723, height: 512, width: 512
find spine area: 5924.5
times:0.15000000000000002
all_path_list length: 1962
0 : 485 (88, 3) -> (572, 2)
1 : 484 (88, 2) -> (571, 2)
2 : 475 (100, 7) -> (574, 1)
******Processing: /nfs3-p1/zsxm/dataset/aorta/fangcaiyu-Im21-91/2/  ******
sequence length: 267, height: 512, width: 512
find spine area: 1856.5
times:0.35
all_path_list length: 1606
0 : 207 (0, 6) -> (206, 6)
1 : 193 (0, 6) -> (192, 15)
2 : 187 (7, 12) -> (193, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/fengyonghua-Im21-89/2/  ******
sequence length: 255, height: 512, width: 512
find spine area: 843.5
times:0.5499999999999999
all_path_list length: 1644
0 : 187 (5, 7) -> (191, 10)
1 : 178 (14, 10) -> (191, 6)
2 : 154 (5, 7) -> (158, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/fuhuajin-Im29-126/2/  ******
sequence length: 666, height: 512, width: 512
find spine area: 2127.0
times:0.44999999999999996
all_path_list length: 2853
0 : 379 (196, 5) -> (574, 7)
1 : 370 (196, 5) -> (565, 5)
2 : 341 (196, 5) -> (536, 11)
all_path_list length: 4519
0 : 481 (185, 11) -> (665, 2)
1 : 381 (185, 11) -> (565, 9)
2 : 352 (185, 11) -> (536, 12)
all_path_list length: 6012
0 : 506 (160, 11) -> (665, 4)
1 : 483 (160, 11) -> (642, 4)
2 : 406 (160, 11) -> (565, 10)
all_path_list length: 7247
0 : 643 (23, 13) -> (665, 6)
1 : 620 (23, 13) -> (642, 7)
2 : 543 (23, 13) -> (565, 10)
******Processing: /nfs3-p1/zsxm/dataset/aorta/gaomingsheng-Im34-151/2/  ******
sequence length: 720, height: 512, width: 512
find spine area: 1020.0
times:0.35
all_path_list length: 2577
0 : 533 (82, 8) -> (614, 10)
1 : 479 (67, 9) -> (545, 16)
2 : 439 (67, 9) -> (505, 7)
******Processing: /nfs3-p1/zsxm/dataset/aorta/gengguorui-Im18-66/2/  ******
sequence length: 128, height: 512, width: 512
find spine area: 737.0
times:0.5499999999999999
all_path_list length: 665
0 : 59 (28, 7) -> (86, 4)
1 : 59 (28, 7) -> (86, 5)
2 : 58 (28, 6) -> (85, 10)
all_path_list length: 1145
0 : 60 (25, 7) -> (84, 11)
1 : 60 (25, 7) -> (84, 12)
2 : 60 (25, 4) -> (84, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/gengyongchao-Im21-83/2/  ******
sequence length: 244, height: 512, width: 512
find spine area: 1458.0
times:0.6
all_path_list length: 1952
0 : 90 (3, 10) -> (92, 10)
1 : 83 (10, 8) -> (92, 10)
2 : 69 (124, 14) -> (192, 6)
all_path_list length: 3428
0 : 92 (1, 15) -> (92, 20)
1 : 90 (3, 13) -> (92, 11)
2 : 86 (3, 13) -> (88, 17)
all_path_list length: 4458
0 : 92 (1, 17) -> (92, 22)
1 : 91 (3, 13) -> (93, 13)
2 : 90 (3, 13) -> (92, 11)
all_path_list length: 5472
0 : 92 (1, 18) -> (92, 23)
1 : 91 (3, 16) -> (93, 13)
2 : 90 (3, 16) -> (92, 11)
all_path_list length: 6533
0 : 92 (1, 24) -> (92, 27)
1 : 91 (3, 21) -> (93, 14)
2 : 90 (3, 21) -> (92, 12)
all_path_list length: 7525
0 : 92 (1, 23) -> (92, 27)
1 : 91 (3, 22) -> (93, 16)
2 : 90 (3, 22) -> (92, 13)
all_path_list length: 8310
0 : 92 (1, 25) -> (92, 27)
1 : 91 (3, 22) -> (93, 16)
2 : 90 (3, 22) -> (92, 13)
all_path_list length: 8692
0 : 92 (1, 28) -> (92, 27)
1 : 91 (3, 22) -> (93, 16)
2 : 90 (3, 22) -> (92, 13)
Can't find aorta!
******Processing: /nfs3-p1/zsxm/dataset/aorta/hebin-Im41-91/2/  ******
sequence length: 270, height: 512, width: 512
find spine area: 1415.0
times:0.3
all_path_list length: 1489
0 : 189 (33, 14) -> (221, 2)
1 : 150 (44, 10) -> (193, 10)
2 : 144 (45, 11) -> (188, 5)
******Processing: /nfs3-p1/zsxm/dataset/aorta/hejianjun-Im31-99/2/  ******
sequence length: 639, height: 512, width: 512
find spine area: 1908.0
times:0.6
all_path_list length: 2945
0 : 544 (20, 11) -> (563, 7)
1 : 532 (32, 18) -> (563, 8)
2 : 500 (32, 18) -> (531, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/heshengnan-Im16-90/2/  ******
sequence length: 253, height: 512, width: 512
find spine area: 549.0
times:0.49999999999999994
all_path_list length: 1870
0 : 185 (2, 4) -> (186, 11)
1 : 180 (2, 4) -> (181, 15)
2 : 179 (3, 5) -> (181, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/huangjianhua-Im38-144/2/  ******
sequence length: 648, height: 512, width: 512
find spine area: 2561.0
times:0.25
all_path_list length: 2087
0 : 315 (236, 3) -> (550, 5)
1 : 310 (242, 1) -> (551, 15)
2 : 250 (242, 1) -> (491, 6)
all_path_list length: 3672
0 : 570 (30, 10) -> (599, 6)
1 : 560 (30, 10) -> (589, 17)
2 : 446 (30, 8) -> (475, 4)
******Processing: /nfs3-p1/zsxm/dataset/aorta/hujiaguo-Im34-58/2/  ******
sequence length: 666, height: 512, width: 512
find spine area: 573.5
times:0.39999999999999997
all_path_list length: 2485
0 : 554 (35, 14) -> (588, 11)
1 : 508 (57, 4) -> (564, 3)
2 : 500 (35, 14) -> (534, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/jiangquanfeng-Im 50-107/2/  ******
sequence length: 537, height: 512, width: 512
find spine area: 1497.0
times:0.6
all_path_list length: 1647
0 : 310 (0, 7) -> (309, 10)
1 : 309 (0, 6) -> (308, 10)
2 : 308 (0, 4) -> (307, 7)
all_path_list length: 2871
0 : 525 (0, 13) -> (524, 5)
1 : 444 (0, 9) -> (443, 8)
2 : 413 (0, 13) -> (412, 10)
******Processing: /nfs3-p1/zsxm/dataset/aorta/litingxian-Im36-141/2/  ******
sequence length: 618, height: 512, width: 512
find spine area: 804.5
times:0.6
all_path_list length: 1899
0 : 137 (423, 10) -> (559, 3)
1 : 136 (423, 10) -> (558, 2)
2 : 126 (108, 2) -> (233, 0)
all_path_list length: 3143
0 : 359 (259, 10) -> (617, 11)
1 : 358 (125, 5) -> (482, 5)
2 : 358 (124, 4) -> (481, 5)
all_path_list length: 5042
0 : 566 (52, 12) -> (617, 15)
1 : 566 (52, 12) -> (617, 18)
2 : 481 (29, 9) -> (509, 13)
******Processing: /nfs3-p1/zsxm/dataset/aorta/luohuafa-im29-96/2/  ******
sequence length: 323, height: 666, width: 512
find spine area: 1336.5
times:0.25
all_path_list length: 1435
0 : 316 (0, 11) -> (315, 9)
1 : 275 (0, 11) -> (274, 9)
2 : 236 (4, 6) -> (239, 6)
******Processing: /nfs3-p1/zsxm/dataset/aorta/lushida-Im33-159/2/  ******
sequence length: 705, height: 512, width: 512
find spine area: 712.5
times:0.39999999999999997
all_path_list length: 2725
0 : 377 (105, 2) -> (481, 10)
1 : 364 (119, 4) -> (482, 5)
2 : 360 (123, 1) -> (482, 6)
all_path_list length: 3914
0 : 389 (105, 2) -> (493, 13)
1 : 375 (119, 4) -> (493, 4)
2 : 372 (123, 1) -> (494, 4)
all_path_list length: 5072
0 : 600 (105, 3) -> (704, 22)
1 : 592 (105, 3) -> (696, 16)
2 : 570 (105, 3) -> (674, 12)
******Processing: /nfs3-p1/zsxm/dataset/aorta/lvpeiji-Im32-136/2/  ******
sequence length: 636, height: 512, width: 512
find spine area: 786.0
times:0.44999999999999996
all_path_list length: 2142
0 : 431 (0, 10) -> (430, 15)
1 : 430 (0, 6) -> (429, 5)
2 : 396 (33, 4) -> (428, 2)
******Processing: /nfs3-p1/zsxm/dataset/aorta/manyanfeng-Im32-153/2/  ******
sequence length: 660, height: 512, width: 512
find spine area: 620.0
times:0.25
all_path_list length: 3071
0 : 367 (0, 9) -> (366, 18)
1 : 365 (0, 9) -> (364, 14)
2 : 359 (0, 9) -> (358, 18)
all_path_list length: 4422
0 : 660 (0, 10) -> (659, 20)
1 : 633 (0, 10) -> (632, 9)
2 : 528 (0, 10) -> (527, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/maoliang-Im12-90/2/  ******
sequence length: 253, height: 512, width: 512
find spine area: 822.5
times:0.2
all_path_list length: 1497
0 : 196 (5, 10) -> (200, 7)
1 : 184 (17, 3) -> (200, 6)
2 : 152 (17, 3) -> (168, 15)
******Processing: /nfs3-p1/zsxm/dataset/aorta/modehe-Im34-89/2/  ******
sequence length: 301, height: 512, width: 512
find spine area: 2515.5
times:0.44999999999999996
all_path_list length: 1424
0 : 265 (36, 18) -> (300, 9)
1 : 264 (37, 15) -> (300, 10)
2 : 264 (37, 14) -> (300, 10)
******Processing: /nfs3-p1/zsxm/dataset/aorta/nibingshan-Im33-42/2/  ******
sequence length: 696, height: 512, width: 512
find spine area: 584.5
times:0.5499999999999999
all_path_list length: 2646
0 : 601 (0, 9) -> (600, 3)
1 : 583 (17, 5) -> (599, 3)
2 : 552 (0, 9) -> (551, 5)
******Processing: /nfs3-p1/zsxm/dataset/aorta/qiquanping-Im25-54/2/  ******
sequence length: 1073, height: 512, width: 512
find spine area: 679.5
times:0.5499999999999999
all_path_list length: 4227
0 : 698 (0, 12) -> (697, 7)
1 : 594 (0, 12) -> (593, 18)
2 : 568 (0, 12) -> (567, 12)
******Processing: /nfs3-p1/zsxm/dataset/aorta/shengfumei-Im31-132/2/  ******
sequence length: 612, height: 512, width: 512
find spine area: 740.5
times:0.3
all_path_list length: 2601
0 : 514 (33, 16) -> (546, 10)
1 : 504 (41, 6) -> (544, 5)
2 : 482 (61, 3) -> (542, 2)
******Processing: /nfs3-p1/zsxm/dataset/aorta/shenhuagou-Im33-95/2/  ******
sequence length: 633, height: 512, width: 512
find spine area: 533.5
times:0.44999999999999996
all_path_list length: 2417
0 : 486 (30, 13) -> (515, 5)
1 : 480 (32, 9) -> (511, 3)
2 : 466 (30, 13) -> (495, 11)
******Processing: /nfs3-p1/zsxm/dataset/aorta/shenjiaren-Im35-136/2/  ******
sequence length: 255, height: 512, width: 512
find spine area: 1455.0
times:0.49999999999999994
all_path_list length: 1768
0 : 161 (23, 5) -> (183, 10)
1 : 152 (32, 10) -> (183, 11)
2 : 113 (23, 5) -> (135, 18)
******Processing: /nfs3-p1/zsxm/dataset/aorta/shenjie-Im36-100/2/  ******
sequence length: 715, height: 512, width: 512
find spine area: 4348.5
times:0.35
all_path_list length: 1839
0 : 298 (298, 5) -> (595, 13)
1 : 274 (298, 5) -> (571, 2)
2 : 272 (300, 4) -> (571, 2)
all_path_list length: 3282
0 : 607 (59, 11) -> (665, 6)
1 : 606 (59, 11) -> (664, 9)
2 : 602 (59, 11) -> (660, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/shiaihua-Im28-108/2/  ******
sequence length: 643, height: 512, width: 512
find spine area: 733.0
times:0.35
all_path_list length: 2599
0 : 427 (42, 8) -> (468, 10)
1 : 416 (42, 8) -> (457, 11)
2 : 395 (43, 4) -> (437, 5)
******Processing: /nfs3-p1/zsxm/dataset/aorta/shilei-Im21-92/2/  ******
sequence length: 1129, height: 512, width: 512
find spine area: 667.0
times:0.6
all_path_list length: 3589
0 : 954 (64, 15) -> (1017, 7)
1 : 952 (64, 6) -> (1015, 4)
2 : 835 (64, 15) -> (898, 4)
******Processing: /nfs3-p1/zsxm/dataset/aorta/shuzhiming-Im28-80/2/  ******
sequence length: 129, height: 512, width: 512
find spine area: 692.5
find spine area: 2.0
times:0.3
all_path_list length: 853
0 : 92 (10, 10) -> (101, 9)
1 : 88 (10, 10) -> (97, 8)
2 : 64 (14, 8) -> (77, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/taofangguo-Im30-142/2/  ******
sequence length: 639, height: 512, width: 512
find spine area: 639.0
times:0.3
all_path_list length: 2749
0 : 406 (41, 7) -> (446, 9)
1 : 393 (36, 14) -> (428, 15)
2 : 390 (40, 14) -> (429, 7)
******Processing: /nfs3-p1/zsxm/dataset/aorta/wangaishui-Im75-128/2/  ******
sequence length: 711, height: 512, width: 512
find spine area: 903.0
times:0.3
all_path_list length: 2157
0 : 680 (20, 9) -> (699, 10)
1 : 539 (20, 9) -> (558, 6)
2 : 492 (89, 7) -> (580, 10)
******Processing: /nfs3-p1/zsxm/dataset/aorta/wanggangxiang-Im16-90/2/  ******
sequence length: 745, height: 512, width: 512
find spine area: 559.0
times:0.5499999999999999
all_path_list length: 2720
0 : 474 (131, 10) -> (604, 6)
1 : 454 (131, 10) -> (584, 8)
2 : 448 (131, 10) -> (578, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/wanggaosheng-Im24-57/2/  ******
sequence length: 135, height: 512, width: 512
find spine area: 906.5
times:0.15000000000000002
all_path_list length: 756
0 : 97 (11, 10) -> (107, 10)
1 : 94 (14, 10) -> (107, 7)
2 : 92 (14, 14) -> (105, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/wangjianjun-Im30-110/2/  ******
sequence length: 630, height: 512, width: 512
find spine area: 3041.5
times:0.3
all_path_list length: 1619
0 : 548 (0, 11) -> (547, 7)
1 : 502 (0, 11) -> (501, 4)
2 : 487 (0, 11) -> (486, 14)
******Processing: /nfs3-p1/zsxm/dataset/aorta/wanglianlian-Im42-139/2/  ******
sequence length: 690, height: 512, width: 512
find spine area: 3090.5
times:0.15000000000000002
all_path_list length: 1705
0 : 269 (279, 1) -> (547, 1)
1 : 268 (281, 4) -> (548, 3)
2 : 249 (281, 4) -> (529, 2)
all_path_list length: 2840
0 : 311 (261, 5) -> (571, 10)
1 : 310 (261, 5) -> (570, 6)
2 : 269 (261, 5) -> (529, 6)
all_path_list length: 4316
0 : 655 (35, 18) -> (689, 25)
1 : 592 (35, 18) -> (626, 11)
2 : 542 (69, 6) -> (610, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/wangmanying-Im23-134/2/  ******
sequence length: 588, height: 512, width: 512
find spine area: 2113.5
find spine area: 590.0
times:0.35
all_path_list length: 2237
0 : 232 (0, 7) -> (231, 9)
1 : 228 (0, 7) -> (227, 10)
2 : 224 (8, 9) -> (231, 6)
all_path_list length: 3385
0 : 439 (0, 7) -> (438, 11)
1 : 412 (0, 7) -> (411, 13)
2 : 409 (0, 7) -> (408, 11)
******Processing: /nfs3-p1/zsxm/dataset/aorta/wangzhongwen-Im36-80/2/  ******
sequence length: 133, height: 512, width: 512
find spine area: 4727.0
times:0.35
all_path_list length: 738
0 : 95 (11, 9) -> (105, 4)
1 : 91 (15, 8) -> (105, 4)
2 : 91 (15, 9) -> (105, 3)
******Processing: /nfs3-p1/zsxm/dataset/aorta/wuguangde-im55-205/2/  ******
sequence length: 299, height: 512, width: 512
find spine area: 2193.5
find spine area: 251.5
find spine area: 156.5
times:0.35
all_path_list length: 1565
0 : 195 (61, 9) -> (255, 4)
1 : 173 (61, 11) -> (233, 4)
2 : 170 (61, 9) -> (230, 12)
all_path_list length: 2517
0 : 202 (59, 14) -> (260, 12)
1 : 201 (61, 21) -> (261, 14)
2 : 177 (59, 14) -> (235, 13)
******Processing: /nfs3-p1/zsxm/dataset/aorta/xiewenbo-Im25-40/2/  ******
sequence length: 1033, height: 512, width: 512
find spine area: 532.5
times:0.6
all_path_list length: 3289
0 : 898 (10, 9) -> (907, 15)
1 : 892 (15, 5) -> (906, 2)
2 : 862 (26, 4) -> (887, 1)
******Processing: /nfs3-p1/zsxm/dataset/aorta/xujinbiao-Im37-146/2/  ******
sequence length: 651, height: 512, width: 512
find spine area: 804.5
times:0.35
all_path_list length: 2794
0 : 510 (75, 9) -> (584, 13)
1 : 438 (75, 9) -> (512, 12)
2 : 387 (75, 9) -> (461, 6)
******Processing: /nfs3-p1/zsxm/dataset/aorta/xujinyang-Im15-30/2/  ******
sequence length: 139, height: 512, width: 512
find spine area: 1226.5
times:0.35
all_path_list length: 889
0 : 94 (10, 7) -> (103, 5)
1 : 93 (10, 7) -> (102, 3)
2 : 91 (10, 10) -> (100, 7)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yangwenbin-Im44-125/2/  ******
sequence length: 699, height: 512, width: 512
find spine area: 537.0
times:0.3
all_path_list length: 1951
0 : 210 (87, 10) -> (296, 10)
1 : 196 (87, 10) -> (282, 13)
2 : 194 (87, 10) -> (280, 7)
all_path_list length: 3241
0 : 243 (23, 8) -> (265, 9)
1 : 235 (23, 8) -> (257, 2)
2 : 234 (23, 8) -> (256, 5)
all_path_list length: 4401
0 : 274 (417, 7) -> (690, 9)
1 : 258 (0, 13) -> (257, 8)
2 : 256 (417, 7) -> (672, 5)
all_path_list length: 5490
0 : 517 (0, 12) -> (516, 17)
1 : 485 (0, 12) -> (484, 22)
2 : 473 (0, 12) -> (472, 16)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yangyoufu-Im16-25/2/  ******
sequence length: 122, height: 512, width: 512
find spine area: 1817.0
times:0.3
all_path_list length: 1084
0 : 65 (0, 8) -> (64, 8)
1 : 60 (5, 11) -> (64, 6)
2 : 58 (7, 15) -> (64, 20)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yaoliang-Im30-83/2/  ******
sequence length: 289, height: 512, width: 512
find spine area: 3250.0
times:0.05000000000000002
all_path_list length: 1549
0 : 88 (89, 9) -> (176, 17)
1 : 81 (132, 13) -> (212, 10)
2 : 79 (89, 9) -> (167, 19)
all_path_list length: 2216
0 : 99 (190, 3) -> (288, 6)
1 : 93 (87, 17) -> (179, 10)
2 : 89 (190, 3) -> (278, 7)
all_path_list length: 2818
0 : 183 (25, 16) -> (207, 7)
1 : 177 (31, 10) -> (207, 6)
2 : 159 (31, 9) -> (189, 8)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yinghao-Im18-86/2/  ******
sequence length: 215, height: 666, width: 512
find spine area: 1762.5
times:0.3
all_path_list length: 1613
0 : 185 (0, 5) -> (184, 4)
1 : 171 (0, 5) -> (170, 14)
2 : 156 (0, 5) -> (155, 15)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yingzhemin-Im58-108/2/  ******
sequence length: 277, height: 512, width: 512
find spine area: 1606.0
times:0.2
all_path_list length: 1364
0 : 196 (11, 10) -> (206, 4)
1 : 195 (11, 10) -> (205, 4)
2 : 152 (125, 5) -> (276, 5)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yuanenchun-Im27-93/2/  ******
sequence length: 279, height: 512, width: 512
find spine area: 1754.0
times:0.2
all_path_list length: 1910
0 : 189 (26, 10) -> (214, 3)
1 : 172 (29, 7) -> (200, 8)
2 : 170 (26, 10) -> (195, 13)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yujinmin-Im25-78/2/  ******
sequence length: 977, height: 512, width: 512
find spine area: 1882.0
times:0.3
all_path_list length: 2561
0 : 738 (15, 3) -> (752, 5)
1 : 703 (15, 3) -> (717, 7)
2 : 700 (53, 8) -> (752, 5)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yukongban-Im30-139/2/  ******
sequence length: 636, height: 512, width: 512
find spine area: 782.0
times:0.49999999999999994
all_path_list length: 2987
0 : 504 (15, 10) -> (518, 4)
1 : 495 (22, 13) -> (516, 0)
2 : 486 (31, 5) -> (516, 0)
******Processing: /nfs3-p1/zsxm/dataset/aorta/yuliyao-Im25-31/2/  ******
sequence length: 672, height: 512, width: 512
find spine area: 993.0
times:0.39999999999999997
all_path_list length: 2724
0 : 451 (35, 5) -> (485, 8)
1 : 449 (36, 5) -> (484, 12)
2 : 448 (36, 5) -> (483, 7)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zengzhengyou-Im54-145/2/  ******
sequence length: 663, height: 512, width: 512
find spine area: 2725.5
times:0.6
all_path_list length: 2158
0 : 226 (268, 2) -> (493, 13)
1 : 226 (38, 6) -> (263, 3)
2 : 222 (268, 2) -> (489, 17)
all_path_list length: 3472
0 : 494 (0, 9) -> (493, 13)
1 : 490 (0, 9) -> (489, 19)
2 : 483 (11, 14) -> (493, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhangxinliang-Im16-24/2/  ******
sequence length: 58, height: 512, width: 512
find spine area: 1193.0
find spine area: 388.0
times:0.49999999999999994
all_path_list length: 449
0 : 51 (7, 13) -> (57, 10)
1 : 51 (7, 8) -> (57, 11)
2 : 27 (31, 10) -> (57, 10)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhangxiyuan-Im9-50/2/  ******
sequence length: 221, height: 512, width: 512
find spine area: 4452.0
times:0.10000000000000002
all_path_list length: 991
0 : 155 (10, 5) -> (164, 10)
1 : 152 (10, 5) -> (161, 6)
2 : 146 (10, 5) -> (155, 11)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhaojianbin-Im31-158/2/  ******
sequence length: 669, height: 512, width: 512
find spine area: 2084.5
times:0.35
all_path_list length: 2890
0 : 250 (0, 11) -> (249, 8)
1 : 240 (0, 11) -> (239, 10)
2 : 225 (0, 11) -> (224, 7)
all_path_list length: 4495
0 : 531 (0, 12) -> (530, 15)
1 : 501 (30, 9) -> (530, 7)
2 : 489 (0, 12) -> (488, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhaojun-Im45-103/2/  ******
sequence length: 632, height: 512, width: 512
find spine area: 17894.5
find spine area: 808.0
find spine area: 17.5
find spine area: 5.5
find spine area: 1.0
find spine area: 25370.0
times:0.10000000000000002
all_path_list length: 2179
0 : 229 (0, 7) -> (228, 3)
1 : 191 (281, 4) -> (471, 8)
2 : 190 (38, 9) -> (227, 3)
all_path_list length: 3573
0 : 251 (0, 12) -> (250, 9)
1 : 251 (0, 12) -> (250, 11)
2 : 225 (262, 3) -> (486, 6)
all_path_list length: 4853
0 : 509 (0, 18) -> (508, 13)
1 : 504 (0, 18) -> (503, 13)
2 : 497 (0, 18) -> (496, 11)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhengsiqing-Im35-106/2/  ******
sequence length: 666, height: 512, width: 512
find spine area: 1991.0
times:0.3
all_path_list length: 3064
0 : 348 (31, 12) -> (378, 9)
1 : 339 (31, 12) -> (369, 12)
2 : 336 (31, 12) -> (366, 11)
all_path_list length: 4558
0 : 368 (245, 24) -> (612, 22)
1 : 368 (245, 24) -> (612, 24)
2 : 366 (31, 13) -> (396, 13)
all_path_list length: 5834
0 : 635 (31, 14) -> (665, 21)
1 : 630 (31, 14) -> (660, 18)
2 : 596 (45, 22) -> (640, 14)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhoufei-Im16-97/2/  ******
sequence length: 581, height: 512, width: 512
find spine area: 1656.0
times:0.49999999999999994
all_path_list length: 3360
0 : 466 (0, 4) -> (465, 2)
1 : 412 (0, 4) -> (411, 6)
2 : 385 (0, 4) -> (384, 9)
******Processing: /nfs3-p1/zsxm/dataset/aorta/zhoujinhai-Im14-94/2/  ******
sequence length: 945, height: 512, width: 512
find spine area: 2807.5
times:0.44999999999999996
all_path_list length: 3558
0 : 875 (0, 9) -> (874, 5)
1 : 874 (0, 5) -> (873, 1)
2 : 821 (0, 9) -> (820, 6)
'''

In [ ]:
'''
for p in range(len(input_folder_list)):
    print(f'******Processing: {input_folder_list[p]}  ******')
    patient = load_scan(input_folder_list[p])
    patient_pixels = get_pixels_hu(patient)
    img_height, img_width = patient_pixels.shape[1:3]
    print(f'sequence length: {patient_pixels.shape[0]}, height: {img_height}, width: {img_width}')
    spine_x, spine_y = 0, 0
    
    patient_sum = np.sum(patient_pixels, axis=0)
    patient_sum = patient_sum.astype(np.float32)
    max_val = patient_sum.max()
    times = 0.6
    while times > 0:
        thr, patient_thr = cv2.threshold(patient_sum, max_val*times, 255, cv2.THRESH_BINARY)
        patient_thr = patient_thr.astype(np.uint8)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        patient_thr = cv2.erode(patient_thr, kernel, iterations=2)
        sum_contours, _ = cv2.findContours(patient_thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        sum_contours_list = [(cv2.contourArea(contour), contour) for contour in sum_contours]
        sum_contours_list.sort(key=lambda x: x[0], reverse=True)
        if len(sum_contours_list) == 0 \
        or len(sum_contours_list) > 1 and sum_contours_list[0][0] < 2 * sum_contours_list[1][0] \
        or sum_contours_list[0][0] < img_height*img_width*0.002:
            times -= 0.05
            continue
        find_flag = False
        for area, contour in sum_contours_list:
            print(f'find spine area: {area}')
            x, y, w, h = cv2.boundingRect(contour)
            cx, cy = x+w//2, y+h//2
            if img_height/2 < cy < img_height*0.75 and img_width*0.375 < cx < img_width*0.625:
                spine_x, spine_y = cx, cy
                find_flag = True
                break
        if find_flag:
            print(f'times:{times}')
            break
        times -= 0.05

    if spine_x == 0 or spine_y == 0:
        raise Exception(f'spine_x:{spine_x}, spine_y:{spine_y}')
            
    all_path_list = []
    for t in range(len(set_window_list)):
        patient_copy = set_window_list[t](patient_pixels, 300, 600)
        start_x = int(spine_x-img_width*0.15)
        end_x = int(spine_x+img_width*0.15)
        start_y = int(spine_y-img_height*0.4)
        end_y = int(spine_y+img_height*0.1)
        patient_cut = patient_copy[:, start_y:end_y, start_x:end_x]

        cut_height, cut_width = patient_cut.shape[1:3]
        cut_min_side = min(cut_height, cut_width)
        pre_contours, _ = cv2.findContours(patient_cut[0], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        pre_circle = list(map(lambda x: cv2.minEnclosingCircle(x), pre_contours))
        path_dict = {}
        for i in range(1, len(patient_cut)):
            cur_contours, _ = cv2.findContours(patient_cut[i], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            cur_circle = list(map(lambda x: cv2.minEnclosingCircle(x), cur_contours))
            for j in range(len(pre_circle)):
                if pre_circle[j][0][0] < 0.4*cut_width \
                or pre_circle[j][0][0] < 0.6*cut_width and pre_circle[j][0][1] > 0.8*cut_height:
                    continue
                candidate_list = []
                for k in range(len(cur_circle)):
                    if cur_circle[k][0][0] < 0.4*cut_width \
                    or cur_circle[k][0][0] < 0.6*cut_width and cur_circle[k][0][1] > 0.8*cut_height:
                        continue
                    dis = sqrt((pre_circle[j][0][0]-cur_circle[k][0][0])**2+(pre_circle[j][0][1]-cur_circle[k][0][1])**2)
                    max_r, min_r = max(pre_circle[j][1], cur_circle[k][1]), min(pre_circle[j][1], cur_circle[k][1])
                    if dis <= max_r:#dis <= max_r - 0.5 * min_r:
                        candidate_list.append((k, cv2.contourArea(cur_contours[k])))

                if len(candidate_list) == 0:
                    continue
                candidate_list.sort(key=lambda x:x[1])
                path = path_dict.pop((i-1, j), Elem((i-1, j), pre_contours[j]))
                temp_path_list = [copy.deepcopy(path) for s in range(len(candidate_list))]
                for s, candidate in enumerate(candidate_list):
                    k = candidate[0]
                    if get_intersection(patient_cut[i], pre_contours[j], cur_contours[k]) < 0.6:
                        continue
                    temp_path_list[s].append((i, k), cur_contours[k])
                    if not (i, k) in path_dict \
                    or len(path_dict[(i, k)]) < len(temp_path_list[s]) \
                    or path_dict[(i, k)].get_prev_area() < temp_path_list[s].get_prev_area():
                        path_dict[(i, k)] = temp_path_list[s]

            pre_contours = cur_contours
            pre_circle = cur_circle

        path_list = list(path_dict.values())
        path_list.sort(key=lambda x: len(x), reverse=True)
        all_path_list += path_list
        
    print('all_path_list length:', len(all_path_list))
    if len(all_path_list) == 0:
        print("Can't find aorta!")
        continue
    all_path_list.sort(key=lambda x: len(x), reverse=True)
    for i in range(min(5, len(all_path_list))):
        print(i, ':', len(all_path_list[i]), all_path_list[i].root, '->', all_path_list[i].end)
        
    path = all_path_list[0]
    start = path.root[0]
    end = path.end[0]+1
    
    path.get_bboxes()
    
    patient = load_scan(input_folder_list[p])
    draw_bbox(patient, path, start_x, start_y, os.path.join(input_folder_list[p], 'save'))
'''